# Data Pipeline - Recon3D metabolic network

## Input Data

'Recon3D_301.mat' and 'Recon3DModel_301.mat' - Go to https://vmh.uni.lu/#downloadview, choose 'Recon3D_3.01', unzip zip file.

## Code

### Initialize Recon3D model

Load Recon3D model

In [ ]:
load('input_data/Recon3D_301.mat');
load('input_data/Recon3DModel_301.mat');
model = Recon3D;

Fix reaction subsystems

In [ ]:
% model.subSystems
for i = 1:length(model.rxns)
    model.subSystems{i} = model.subSystems{i}{1};
end

Clear objective values

In [ ]:
model.c = zeros(size(model.c));

Clear dm/dt

In [ ]:
model.b = zeros(size(model.b));

### Remove erroneous energy-generating cycles

#### Implement bounds from Recon3DModel

In [ ]:
for i = 1:length(model.rxns)
    if any(strcmp(Recon3DModel.rxns,model.rxns{i}))
        model.lb(i) = Recon3DModel.lb(strcmp(Recon3DModel.rxns,model.rxns{i}));
        model.ub(i) = Recon3DModel.ub(strcmp(Recon3DModel.rxns,model.rxns{i}));
    else
        model.lb(i) = 0;
        model.ub(i) = 0;
    end
end

#### Add back bounds from Recon3D if doesn't cause erroneous energy-generating cycles

Erroneous energy-generating cycles to test (from Fritzemeier et al):<br>
atp[c] + h2o[c] --> adp[c] + h[c] + pi[c]<br>
ctp[c] + h2o[c] --> cdp[c] + h[c] + pi[c]<br>
gtp[c] + h2o[c] --> gdp[c] + h[c] + pi[c]<br>
utp[c] + h2o[c] --> udp[c] + h[c] + pi[c]<br>
itp[c] + h2o[c] --> idp[c] + h[c] + pi[c]<br>
nadh[c] --> h[c] + nad[c]<br>
nadph[c] --> h[c] + nadp[c]<br>
fadh2[c] --> 2 h[c] + fad[c]<br>
q10h2[c] --> 2 h[c] + q10[c]<br>
h2o[c] + accoa[c] --> h[c] + ac[c] + coa[c]<br>
glu_L[c] + h2o[c] --> akg[c] + nh4[c] + h[c]<br>
h[i] --> h[c]<br>

Restrict exchange fluxes to 0, all other fluxes to +/- 1

In [ ]:
for i = 1:length(model.rxns)
    if (model.lb(i) ~= 0) || (model.ub(i) ~= 0)
        if strcmp(model.subSystems{i},'Exchange/demand reaction')
            if length(model.rxns{i}) >= 3
                if strcmp(model.rxns{i}(1:3),'EX_')
                    model.lb(i) = 0;
                    model.ub(i) = 0;
                else
                    model.lb(i) = 0;
                    model.ub(i) = 0;
                end
            else
                model.lb(i) = 0;
                model.ub(i) = 0;
            end     
        else
            if model.lb(i) < 0
                model.lb(i) = -1;
            else
                model.lb(i) = 0;
            end
            if model.ub(i) > 0
                model.ub(i) = 1;
            else
                model.ub(i) = 0;
            end
        end
    end
end

Add reactions sequentually, keep if still no erroneous energy-generating cycles

In [ ]:
for i = 1:length(model.rxns)
    
    % if not exchange reaction
    if ~strcmp(model.subSystems{i},'Exchange/demand reaction')
    
        % forward direction
        if (model.ub(i) == 0) && (Recon3D.ub(i) > 0)
            
            % set new upper bound
            model.ub(i) = 1;
            model_keep = model;
            
            % test dissipation reactions
            objectives = [];

            % atp
            model.rxns{end+1} = 'energy_dissipation_atp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'atp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'adp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - ATP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % ctp
            model.rxns{end+1} = 'energy_dissipation_ctp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'ctp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'cdp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - CTP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % gtp
            model.rxns{end+1} = 'energy_dissipation_gtp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'gtp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'gdp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - GTP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % utp
            model.rxns{end+1} = 'energy_dissipation_utp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'utp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'udp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - UTP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % itp
            model.rxns{end+1} = 'energy_dissipation_itp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'itp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'idp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - ITP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % nadh
            model.rxns{end+1} = 'energy_dissipation_nadh';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'nadh[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'nad[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - NADH';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % nadph
            model.rxns{end+1} = 'energy_dissipation_nadph';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - NADPH';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % fadh2
            model.rxns{end+1} = 'energy_dissipation_fadh2';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'fadh2[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 2;
            model.S(strcmp(model.mets,'fad[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - FADH2';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % q10h2
            model.rxns{end+1} = 'energy_dissipation_q10h2';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'q10h2[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 2;
            model.S(strcmp(model.mets,'q10[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - Q10H2';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % accoa
            model.rxns{end+1} = 'energy_dissipation_accoa';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'accoa[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'ac[c]'),end) = 1;
            model.S(strcmp(model.mets,'coa[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - ACCOA';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % glu
            model.rxns{end+1} = 'energy_dissipation_glu';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'glu_L[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'akg[c]'),end) = 1;
            model.S(strcmp(model.mets,'nh4[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 2;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - GLU';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % proton
            model.rxns{end+1} = 'energy_dissipation_proton';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'h[i]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - PROTON';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;
            
            % keep only if all objective values are still 0
            if sum(objectives == 0) ~= length(objectives)
                model_keep.ub(i) = 0;
            end
            model = model_keep;
            
        end

        % reverse direction
        if (model.lb(i) == 0) && (Recon3D.lb(i) < 0)
            
            % set new upper bound
            model.lb(i) = -1;
            model_keep = model;
            
            % test dissipation reactions
            objectives = [];

            % atp
            model.rxns{end+1} = 'energy_dissipation_atp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'atp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'adp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - ATP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % ctp
            model.rxns{end+1} = 'energy_dissipation_ctp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'ctp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'cdp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - CTP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % gtp
            model.rxns{end+1} = 'energy_dissipation_gtp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'gtp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'gdp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - GTP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % utp
            model.rxns{end+1} = 'energy_dissipation_utp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'utp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'udp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - UTP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % itp
            model.rxns{end+1} = 'energy_dissipation_itp';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'itp[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'idp[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'pi[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - ITP';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % nadh
            model.rxns{end+1} = 'energy_dissipation_nadh';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'nadh[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'nad[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - NADH';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % nadph
            model.rxns{end+1} = 'energy_dissipation_nadph';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - NADPH';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % fadh2
            model.rxns{end+1} = 'energy_dissipation_fadh2';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'fadh2[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 2;
            model.S(strcmp(model.mets,'fad[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - FADH2';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % q10h2
            model.rxns{end+1} = 'energy_dissipation_q10h2';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'q10h2[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 2;
            model.S(strcmp(model.mets,'q10[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - Q10H2';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % accoa
            model.rxns{end+1} = 'energy_dissipation_accoa';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'accoa[c]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.S(strcmp(model.mets,'ac[c]'),end) = 1;
            model.S(strcmp(model.mets,'coa[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - ACCOA';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % glu
            model.rxns{end+1} = 'energy_dissipation_glu';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'glu_L[c]'),end) = -1;
            model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
            model.S(strcmp(model.mets,'akg[c]'),end) = 1;
            model.S(strcmp(model.mets,'nh4[c]'),end) = 1;
            model.S(strcmp(model.mets,'h[c]'),end) = 2;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - GLU';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;

            % proton
            model.rxns{end+1} = 'energy_dissipation_proton';
            model.S(:,end+1) = zeros(length(model.mets),1);
            model.S(strcmp(model.mets,'h[i]'),end) = -1;
            model.S(strcmp(model.mets,'h[c]'),end) = 1;
            model.lb(end+1) = 0;
            model.ub(end+1) = 9999999;
            model.c(end+1) = 1;
            model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
            model.rules{end+1} = '';
            model.grRules{end+1} = '';
            model.subSystems{end+1} = 'Energy dissipation reaction';
            model.rxnNames{end+1} = 'Energy dissipation - PROTON';
            model.rxnKEGGID{end+1} = '';
            model.rxnKeggOrthology{end+1} = '';
            model.rxnConfidenceScores(end+1) = 0';
            model.rxnReferences{end+1} = '';
            model.rxnECNumbers{end+1} = '';
            model.rxnNotes{end+1} = '';
            model.rxnCOG{end+1} = '';
            model.rxnReconMap{end+1} = '';

            model.A = model.S;
            model.rhs = model.b;
            model.obj = model.c;
            model.sense = repmat('=',1,length(model.mets));
            model.vtype = repmat('C',1,length(model.rxns));
            model.varnames = model.rxns;
            model.modelsense = 'max';

            params.outputflag = 0;
            result = gurobi(model,params);
            objectives(end+1) = result.objval;

            model.c(end) = 0;
            model.lb(end) = 0;
            model.ub(end) = 0;
            
            % keep only if all objective values are still 0
            if sum(objectives == 0) ~= length(objectives)
                model_keep.lb(i) = 0;
            end
            model = model_keep;

        end
    end
end

Reset flux ranges

In [ ]:
% reset flux ranges
for i = 1:length(model.rxns)
    if strcmp(model.rxns{i},'biomass_reaction')
        model.lb(i) = 0;
        model.ub(i) = 9999999;
    elseif length(model.rxns{i}) >= 3
        if strcmp(model.rxns{i}(1:3),'EX_')
            model.lb(i) = -9999999;
            model.ub(i) = 9999999;
        else
            if model.lb(i) < 0
                model.lb(i) = -9999999;
            end
            if model.ub(i) > 0
                model.ub(i) = 9999999;
            end
        end
    else
        if model.lb(i) < 0
            model.lb(i) = -9999999;
        end
        if model.ub(i) > 0
            model.ub(i) = 9999999;
        end
    end
end

### Add/change redox-related reaction information in Recon3D

Split reaction FTHFDH into two separate cytosolic and mitochondrial reactions

In [ ]:
% fix cytosolic reaction
i = find(strcmp(model.rxns,'FTHFDH'));
model.rxnGeneMat(i,strcmp(model.genes,'160428.1')) = 0;
model.rules{i} = '(x(615))';
model.grRules{i} = '(10840.1)';

% create new mitochondrial reaction
model.rxns{end+1} = 'FTHFDHm';
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'10fthf[m]'),end) = -1;
model.S(strcmp(model.mets,'h2o[m]'),end) = -1;
model.S(strcmp(model.mets,'nadp[m]'),end) = -1;
model.S(strcmp(model.mets,'h[m]'),end) = 1;
model.S(strcmp(model.mets,'nadph[m]'),end) = 1;
model.S(strcmp(model.mets,'co2[m]'),end) = 1;
model.S(strcmp(model.mets,'thf[m]'),end) = 1;
model.lb(end+1) = model.lb(i);
model.ub(end+1) = model.ub(i);
model.c(end+1) = model.c(i);
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,strcmp(model.genes,'160428.1')) = 1;
model.rules{end+1} = '(x(626))';
model.grRules{end+1} = '(160428.1)';
model.subSystems{end+1} = model.subSystems{i};
model.rxnNames{end+1} = model.rxnNames{i};
model.rxnKEGGID{end+1} = model.rxnKEGGID{i};
model.rxnKeggOrthology{end+1} = model.rxnKeggOrthology{i};
model.rxnConfidenceScores(end+1) = model.rxnConfidenceScores(i);
model.rxnReferences{end+1} = model.rxnReferences{i};
model.rxnECNumbers{end+1} = model.rxnECNumbers{i};
model.rxnNotes{end+1} = model.rxnNotes{i};
model.rxnCOG{end+1} = model.rxnCOG{i};
model.rxnReconMap{end+1} = model.rxnReconMap{i};

Remove gene MTHFR from reaction MTHFD

In [ ]:
i = find(strcmp(model.rxns,'MTHFD'));
model.rxnGeneMat(i,strcmp(model.genes,'4524.1')) = 0;
model.rules{i} = '(x(628))';
model.grRules{i} = '(4522.1)';

Add reactions catalyzed by NOX isoforms

In [ ]:
% NOX1/2/3/5, transfer from cytoplasm to extracellular
model.rxns{end+1} = 'NOXce';
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
model.S(strcmp(model.mets,'o2[c]'),end) = -2;
model.S(strcmp(model.mets,'o2s[e]'),end) = 2;
model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(:,(end+1):(end+4)) = zeros(length(model.rxns),4);
model.rxnGeneMat(end,(end-3):end) = 1;
model.rules{end+1} = '(x(2249)) | (x(2250)) | (x(2251)) | (x(2252))';
model.genes{end+1} = '27035.1';
model.genes{end+1} = '1536.1';
model.genes{end+1} = '50508.1';
model.genes{end+1} = '79400.1';
model.grRules{end+1} = '(27035.1) or (1536.1) or (50508.1) or (79400.1)';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnNames{end+1} = 'NADPH oxidase, cytoplasm to extracellular';
model.rxnKEGGID{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnConfidenceScores(end+1) = 0;
model.rxnReferences{end+1} = '';
model.rxnECNumbers{end+1} = '1.6.3.1';
model.rxnNotes{end+1} = '';
model.rxnCOG{end+1} = '';
model.rxnReconMap{end+1} = '';

% NOX4, transfer from nucleus to cytoplasm
model.mets{end+1} = 'nadph[n]';
model.S(end+1,:) = zeros(1,length(model.rxns));
model.metNames{end+1} = model.metNames{strcmp(model.mets,'nadph[c]')};
model.metFormulas{end+1} = model.metFormulas{strcmp(model.mets,'nadph[c]')};
model.metCharges(end+1) = model.metCharges(strcmp(model.mets,'nadph[c]'));
model.metCHEBIID{end+1} = model.metCHEBIID{strcmp(model.mets,'nadph[c]')};
model.metKEGGID{end+1} = model.metKEGGID{strcmp(model.mets,'nadph[c]')};
model.metPubChemID{end+1} = model.metPubChemID{strcmp(model.mets,'nadph[c]')};
model.metInChIString{end+1} = model.metInChIString{strcmp(model.mets,'nadph[c]')};
model.metHMDBID{end+1} = model.metHMDBID{strcmp(model.mets,'nadph[c]')};
model.metSmiles{end+1} = model.metSmiles{strcmp(model.mets,'nadph[c]')};
model.metPdMap{end+1} = model.metPdMap{strcmp(model.mets,'nadph[c]')};
model.metReconMap{end+1} = model.metReconMap{strcmp(model.mets,'nadph[c]')};
model.b(end+1) = model.b(strcmp(model.mets,'nadph[c]'));
model.csense(end+1) = 'E';

model.mets{end+1} = 'nadp[n]';
model.S(end+1,:) = zeros(1,length(model.rxns));
model.metNames{end+1} = model.metNames{strcmp(model.mets,'nadp[c]')};
model.metFormulas{end+1} = model.metFormulas{strcmp(model.mets,'nadp[c]')};
model.metCharges(end+1) = model.metCharges(strcmp(model.mets,'nadp[c]'));
model.metCHEBIID{end+1} = model.metCHEBIID{strcmp(model.mets,'nadp[c]')};
model.metKEGGID{end+1} = model.metKEGGID{strcmp(model.mets,'nadp[c]')};
model.metPubChemID{end+1} = model.metPubChemID{strcmp(model.mets,'nadp[c]')};
model.metInChIString{end+1} = model.metInChIString{strcmp(model.mets,'nadp[c]')};
model.metHMDBID{end+1} = model.metHMDBID{strcmp(model.mets,'nadp[c]')};
model.metSmiles{end+1} = model.metSmiles{strcmp(model.mets,'nadp[c]')};
model.metPdMap{end+1} = model.metPdMap{strcmp(model.mets,'nadph[c]')};
model.metReconMap{end+1} = model.metReconMap{strcmp(model.mets,'nadp[c]')};
model.b(end+1) = model.b(strcmp(model.mets,'nadp[c]'));
model.csense(end+1) = 'E';

model.rxns{end+1} = 'NOXnc';
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'nadph[n]'),end) = -1;
model.S(strcmp(model.mets,'o2[n]'),end) = -2;
model.S(strcmp(model.mets,'o2s[c]'),end) = 2;
model.S(strcmp(model.mets,'nadp[n]'),end) = 1;
model.S(strcmp(model.mets,'h[n]'),end) = 1;
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(:,end+1) = zeros(length(model.rxns),1);
model.rxnGeneMat(end,end) = 1;
model.rules{end+1} = '(x(2253))';
model.genes{end+1} = '50507.1';
model.grRules{end+1} = '(50507.1)';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnNames{end+1} = 'NADPH oxidase, nucleus to cytoplasm';
model.rxnKEGGID{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnConfidenceScores(end+1) = 0;
model.rxnReferences{end+1} = '';
model.rxnECNumbers{end+1} = '1.6.3.1';
model.rxnNotes{end+1} = '';
model.rxnCOG{end+1} = '';
model.rxnReconMap{end+1} = '';

% NOX4, transfer from endoplasmic reticulum to cytoplasm
model.rxns{end+1} = 'NOXrc';
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'nadph[r]'),end) = -1;
model.S(strcmp(model.mets,'o2[r]'),end) = -2;
model.S(strcmp(model.mets,'o2s[c]'),end) = 2;
model.S(strcmp(model.mets,'nadp[r]'),end) = 1;
model.S(strcmp(model.mets,'h[r]'),end) = 1;
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,end) = 1;
model.rules{end+1} = '(x(2253))';
model.grRules{end+1} = '(50507.1)';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnNames{end+1} = 'NADPH oxidase, endoplasmic reticulum to cytoplasm';
model.rxnKEGGID{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnConfidenceScores(end+1) = 0;
model.rxnReferences{end+1} = '';
model.rxnECNumbers{end+1} = '1.6.3.1';
model.rxnNotes{end+1} = '';
model.rxnCOG{end+1} = '';
model.rxnReconMap{end+1} = '';

### Fix aspects of model variable in order to run successfully

Remove EC numbers for transport reactions (many incorrect)

In [ ]:
for i = 1:length(model.rxns)
    if any(strcmp({'Transport, endoplasmic reticular','Transport, extracellular','Transport, golgi apparatus','Transport, lysosomal','Transport, mitochondrial','Transport, nuclear','Transport, peroxisomal'},model.subSystems{i}))
        model.rxnECNumbers{i} = '';
    end
end

Remove gene '0' and any associations with Recon3D reactions

In [ ]:
model.rules{strcmp(model.rxns,'HDCA24Gtr')} = '';
model.grRules{strcmp(model.rxns,'HDCA24Gtr')} = '';

i = find(strcmp(model.genes,'0'));
for j = (i+1):(length(model.genes))
    for k = find(model.rxnGeneMat(:,j))'
        model.rules{k} = strrep(model.rules{k},sprintf('x(%d)',j),sprintf('x(%d)',j-1));
    end
end
model.genes(i) = [];
model.rxnGeneMat(:,i) = [];

Remove gene '100507855' and any associations with Recon3D reactions

In [ ]:
model.rules{strcmp(model.rxns,'r0345')} = 'x(161) | x(1877) | x(1876) | x(165) | x(160)';
model.grRules{strcmp(model.rxns,'r0345')} = '204.1 or 203.1 or 122481.1 or 26289.1 or 205.1';

i = find(strcmp(model.genes,'100507855.1'));
for j = (i+1):(length(model.genes))
    for k = find(model.rxnGeneMat(:,j))'
        model.rules{k} = strrep(model.rules{k},sprintf('x(%d)',j),sprintf('x(%d)',j-1));
    end
end
model.genes(i) = [];
model.rxnGeneMat(:,i) = [];

Remove gene '8041' and any associations with Recon3D reactions

In [ ]:
model.rules{strcmp(model.rxns,'HMR_4790')} = '';
model.grRules{strcmp(model.rxns,'HMR_4790')} = '';

i = find(strcmp(model.genes,'8041.1'));
for j = (i+1):(length(model.genes))
    for k = find(model.rxnGeneMat(:,j))'
        model.rules{k} = strrep(model.rules{k},sprintf('x(%d)',j),sprintf('x(%d)',j-1));
    end
end
model.genes(i) = [];
model.rxnGeneMat(:,i) = [];

Rewrite 'model.rules' to fix any original errors

In [ ]:
for i = 1:length(model.rxns)
    genes = {};
    geneids = {};
    isoforms = [];
    lengths = [];
    for j = find(model.rxnGeneMat(i,:))
        genes{end+1} = model.genes{j};
        geneids{end+1} = num2str(j);
        s = strsplit(model.genes{j},'.');
        isoforms(end+1) = str2num(s{2});
        lengths(end+1) = length(model.genes{j});
    end
    [~,index] = sort(lengths,'descend');
    genes = genes(index);
    geneids = geneids(index);
    isoforms = isoforms(index);
    
    [~,index] = sort(isoforms,'descend');
    genes = genes(index);
    geneids = geneids(index);
    
    model.rules{i} = model.grRules{i};
    for j = 1:length(genes)
        model.rules{i} = strrep(model.rules{i},genes{j},sprintf('x(%s)',geneids{j}));
    end
    model.rules{i} = strrep(strrep(model.rules{i},'or','|'),'and','&');
end

Remove gene isoform information - genes with different isoforms are combined into one gene ID

In [ ]:
% new list of genes without isoforms
genes = {};
for i = 1:length(model.genes)
    s = strsplit(model.genes{i},'.');
    if ~any(strcmp(genes,s{1}))
        genes{end+1} = s{1};
    end
end
genes = sort(genes);

% rewrite model.rules and model.rxnGeneMat for new gene list without isoforms
rxnGeneMat = zeros(length(model.rxns),length(genes));
for i = 1:length(model.rxns)
    for j = find(model.rxnGeneMat(i,:))
        s = strsplit(model.genes{j},'.');
        model.rules{i} = strrep(model.rules{i},sprintf('x(%d)',j),sprintf('x(%d)',find(strcmp(genes,s{1}))));
        rxnGeneMat(i,find(strcmp(genes,s{1}))) = 1;
    end
end
model.rxnGeneMat = rxnGeneMat;
model.genes = genes;

for i = 1:length(model.rxns)
    
    % if has rule
    if ~isempty(model.rules{i})
        rule = model.rules{i};
        
        % insert symbolic variables
        s = {};
        for j = find(model.rxnGeneMat(i,:))
            s{end+1} = sprintf('x%d',j);
            rule = strrep(rule,sprintf('x(%d)',j),sprintf('x%d',j));
        end
        
        % create symbolic variables
        for j = 1:length(s)
            syms (s{j});
        end
        
        % simplify grRule
        rule = char(simplify(sym(rule)));
        
        % insert gene variables
        for j = sort(find(model.rxnGeneMat(i,:)),'descend')
            rule = strrep(rule,sprintf('x%d',j),sprintf('x(%d)',j));
        end
        
        % replace rules
        model.rules{i} = rule;
        
        % replace grRules
        for j = sort(find(model.rxnGeneMat(i,:)),'descend')
            rule = strrep(strrep(strrep(rule,sprintf('x(%d)',j),sprintf('(%s)',model.genes{j})),'|','or'),'&','and');
        end
        model.grRules{i} = rule;
        
    end
end

% rewrite model.rxnGeneMat with simplified gene rules
model.rxnGeneMat = zeros(length(model.rxns),length(genes));
for i = 1:length(model.rxns)
    for j = 1:length(model.genes)
        if contains(model.rules{i},sprintf('x(%d)',j))
            model.rxnGeneMat(i,j) = 1;
        end
    end   
end

% remove genes no longer in GPR rules
gene_length_original = length(model.genes);
genes_to_remove = sort(find(sum(model.rxnGeneMat)==0),'descend');
for i = genes_to_remove
    model.genes(i) = [];
    model.rxnGeneMat(:,i) = [];    
    for j = (i+1):gene_length_original
        for k = 1:length(model.rxns)
            model.rules{k} = strrep(model.rules{k},sprintf('x(%d)',j),sprintf('x(%d)',j-1));
        end
    end
end

### Fix metabolite and reaction ID's

Fix metabolite CHEBI ID's

In [ ]:
for i = 1:length(model.mets)
    if length(model.metCHEBIID{i}) > 6
        if strcmpi(model.metCHEBIID{i}(1:6),'CHEBI:')
            model.metCHEBIID{i} = model.metCHEBIID{i}(7:end);
        end
    end 
end

Fix reaction EC numbers

In [ ]:
for i = 1:length(model.rxns)
    
    % if has ec number
    if ~isempty(model.rxnECNumbers{i})
        
        % if TCDB number, remove
        if strcmp(model.rxnECNumbers{i}(1:5),'TCDB:')
            model.rxnECNumbers{i} = '';
            
        % if spontaneous, remove
        elseif any(strcmp({'NonEnzymatic','Spontaneous','spontaneous'},model.rxnECNumbers{i}))
            model.rxnECNumbers{i} = '';
        
        else
            
            % split on "," and ";"
            ecs = strsplit(model.rxnECNumbers{i},{',',';','or',' , ',' ; ',' or ',' '});
            
            % iterate over individual ec's
            to_remove = [];
            for j = 1:length(ecs)
                
                % if empty, remove
                if isempty(ecs{j})
                    to_remove(end+1) = j;
                
                % if starts with "EC:", remove "EC:"
                elseif strcmp(ecs{j}(1:3),'EC:')
                    ecs{j} = ecs{j}(4:end);
                end
                
            end
            
            % remove any empty ec's
            for j = sort(to_remove,'descend')
                ecs(j) = [];
            end
            
            % see if can remove any general ec's that contain more specific ec
            % eg. can remove 1.2.3.- if also contains 1.2.3.4
            % also removes duplicate ec's
            dash1_index = []; dash1_val1 = {};
            dash2_index = []; dash2_val1 = {}; dash2_val2 = {};
            dash3_index = []; dash3_val1 = {}; dash3_val2 = {}; dash3_val3 = {};
            nodash_index = []; nodash_val1 = {}; nodash_val2 = {}; nodash_val3 = {}; nodash_val4 = {};
            for j = 1:length(ecs)
                split = strsplit(ecs{j},'.');
                if strcmp(split{2},'-')
                    dash1_index(end+1) = j;
                    dash1_val1{end+1} = split{1};
                elseif strcmp(split{3},'-')
                    dash2_index(end+1) = j;
                    dash2_val1{end+1} = split{1};
                    dash2_val2{end+1} = split{2};
                elseif strcmp(split{4},'-')
                    dash3_index(end+1) = j;
                    dash3_val1{end+1} = split{1};
                    dash3_val2{end+1} = split{2};
                    dash3_val3{end+1} = split{3};
                else
                    nodash_index(end+1) = j;
                    nodash_val1{end+1} = split{1};
                    nodash_val2{end+1} = split{2};
                    nodash_val3{end+1} = split{3};
                    nodash_val4{end+1} = split{4};
                end
            end
            
            to_remove = [];
            
            for j = 1:length(dash1_index)
                for k = (j+1):length(dash1_index)
                    if strcmp(dash1_val1{j},dash1_val1{k})
                        if ~any(to_remove == dash1_index(j))
                            to_remove(end+1) = dash1_index(j);
                        end
                    end
                end
                for k = 1:length(dash2_index)
                    if strcmp(dash1_val1{j},dash2_val1{k})
                        if ~any(to_remove == dash1_index(j))
                            to_remove(end+1) = dash1_index(j);
                        end
                    end
                end
                for k = 1:length(dash3_index)
                    if strcmp(dash1_val1{j},dash3_val1{k})
                        if ~any(to_remove == dash1_index(j))
                            to_remove(end+1) = dash1_index(j);
                        end
                    end
                end
                for k = 1:length(nodash_index)
                    if strcmp(dash1_val1{j},nodash_val1{k})
                        if ~any(to_remove == dash1_index(j))
                            to_remove(end+1) = dash1_index(j);
                        end
                    end
                end
            end
            
            for j = 1:length(dash2_index)
                for k = (j+1):length(dash2_index)
                    if strcmp(dash2_val1{j},dash2_val1{k}) && strcmp(dash2_val2{j},dash2_val2{k})
                        if ~any(to_remove == dash2_index(j))
                            to_remove(end+1) = dash2_index(j);
                        end
                    end
                end
                for k = 1:length(dash3_index)
                    if strcmp(dash2_val1{j},dash3_val1{k}) && strcmp(dash2_val2{j},dash3_val2{k})
                        if ~any(to_remove == dash2_index(j))
                            to_remove(end+1) = dash2_index(j);
                        end
                    end
                end
                for k = 1:length(nodash_index)
                    if strcmp(dash2_val1{j},nodash_val1{k}) && strcmp(dash2_val2{j},nodash_val2{k})
                        if ~any(to_remove == dash2_index(j))
                            to_remove(end+1) = dash2_index(j);
                        end
                    end
                end
            end
            
            for j = 1:length(dash3_index)
                for k = (j+1):length(dash3_index)
                    if strcmp(dash3_val1{j},dash3_val1{k}) && strcmp(dash3_val2{j},dash3_val2{k}) && strcmp(dash3_val3{j},dash3_val3{k})
                        if ~any(to_remove == dash3_index(j))
                            to_remove(end+1) = dash3_index(j);
                        end
                    end
                end
                for k = 1:length(nodash_index)
                    if strcmp(dash3_val1{j},nodash_val1{k}) && strcmp(dash3_val2{j},nodash_val2{k}) && strcmp(dash3_val3{j},nodash_val3{k})
                        if ~any(to_remove == dash3_index(j))
                            to_remove(end+1) = dash3_index(j);
                        end
                    end
                end
            end
            
            for j = 1:length(nodash_index)
                for k = (j+1):length(nodash_index)
                    if strcmp(nodash_val1{j},nodash_val1{k}) && strcmp(nodash_val2{j},nodash_val2{k}) && strcmp(nodash_val3{j},nodash_val3{k}) && strcmp(nodash_val4{j},nodash_val4{k})
                        if ~any(to_remove == nodash_index(j))
                            to_remove(end+1) = nodash_index(j);
                        end
                    end
                end
            end
        
            for j = sort(to_remove,'descend')
                ecs(j) = [];
            end

            % create new ec number string
            model.rxnECNumbers{i} = strjoin(ecs,' | ');
            
        end
    end
end

### Obtain gene symbols for each gene ID

In [ ]:
% initialize list of gene symbols
model.geneSymbols = {};

% get conversions
% split into groups of 300 genes
for i = 1:floor(length(genes)/300)
    conversion = webread(sprintf('https://biodbnet-abcc.ncifcrf.gov/webServices/rest.php/biodbnetRestApi.json?method=db2db&input=geneid&inputValues=%s&outputs=genesymbol&taxonId=9606&format=row',strjoin(model.genes((300*(i-1)+1):(300*i)),',')));
    for j = 1:length(conversion)
        model.geneSymbols{end+1} = conversion(j).GeneSymbol;
    end
    pause(10)
end
conversion = webread(sprintf('https://biodbnet-abcc.ncifcrf.gov/webServices/rest.php/biodbnetRestApi.json?method=db2db&input=geneid&inputValues=%s&outputs=genesymbol&taxonId=9606&format=row',strjoin(model.genes((300*floor(length(genes)/300)+1):end),',')));
for j = 1:length(conversion)
    model.geneSymbols{end+1} = conversion(j).GeneSymbol;
end

model.geneSymbols = model.geneSymbols';

### Remove EC numbers for transport reactions

Many EC numbers for transport reactions are incorrect

In [ ]:
for i = 1:length(model.rxns)
    if any(strcmp({'Transport, endoplasmic reticular','Transport, extracellular','Transport, golgi apparatus','Transport, lysosomal','Transport, mitochondrial','Transport, nuclear','Transport, peroxisomal'},model.subSystems{i}))
        model.rxnECNumbers{i} = '';
    end
end

### Add KEGG ID's for reactions

Load reactions json file

In [ ]:
data = fileread('../deltag/input_data/reactions.json');
data = jsondecode(data);

Add available KEGG ID's

In [ ]:
for i = 1:length(data.results)
    temp = data.results(i);
    rxn = temp.abbreviation;
    keggid = temp.keggId;
    if ~strcmp(keggid,'') && length(keggid)>0 && sum(strcmp(model.rxns,rxn))==1  
        model.rxnKEGGID{strcmp(model.rxns,rxn)} = keggid;
    end
end

### Remove reactions for accurate results

Removing the following subsystems accurately prevents de novo production of essential amino acids: 'Protein assembly', 'Protein degradation', 'Protein modification', 'Protein formation'

In [ ]:
subsystems_to_remove = {'Protein assembly','Protein degradation','Protein modification','Protein formation'};
for i = 1:length(model.rxns)
    if any(strcmp(subsystems_to_remove,model.subSystems{i}))
        model.lb(i) = 0;
        model.ub(i) = 0;
    end
end

Remove bile acid synthesis

In [ ]:
for i = 1:length(model.rxns)
    if strcmp(model.subSystems{i},'Bile acid synthesis')
        model.lb(i) = 0;
        model.ub(i) = 0;
    end
end

Remove NQO1 futile cycle reactions

In [ ]:
for i = 1:length(model.rxns)
    if any(strcmp({'NADPQNOXR','NADQNOXR','HMR_9538','HMR_9722'},model.rxns{i}))
        model.lb(i) = 0;
        model.ub(i) = 0;
    end
end

Remove oxalosuccinic acid-producing IDH1 reactions

In [ ]:
for i = 1:length(model.rxns)
    if any(strcmp({'r0423','r0424','r0425','r0422'},model.rxns{i}))
        model.lb(i) = 0;
        model.ub(i) = 0;
    end
end

### Remove duplicate reactions

Some reactions exist where stoichiometric vector is an exact multiple of that of other reaction

In [ ]:
reactions_to_remove = {'HMR_7257','PEHSFABPe','G6PDH2c','GNDc','PGLc','RPEc'};
for i = 1:length(model.rxns)
    if any(strcmp(reactions_to_remove,model.rxns{i}))
        model.lb(i) = 0;
        model.ub(i) = 0;
    end
end

### Cleanup

Write reaction formulas

In [ ]:
% reaction formulas
formulas = {};
for i = 1:length(model.rxns)
    
    % initialize formula
    formulas{end+1} = '';
    
    % substrates
    for j = find(model.S(:,i)<0)'
        if isempty(formulas{end})
            formulas{end} = sprintf('%d %s',-full(model.S(j,i)),model.mets{j});
        else
            formulas{end} = [formulas{end},' + ',sprintf('%d %s',-full(model.S(j,i)),model.mets{j})];
        end
    end
    
    % reaction arrow
    if (model.lb(i) < 0) && (model.ub(i) > 0)
        formulas{end} = [formulas{end},' <=> '];
    elseif (model.lb(i) < 0)
        formulas{end} = [formulas{end},' <-- '];
    elseif (model.ub(i) > 0)
        formulas{end} = [formulas{end},' --> '];
    else
        formulas{end} = [formulas{end},' X--X '];
    end
    
    % products
    numproducts = 0;
    for j = find(model.S(:,i)>0)'
        if numproducts == 0
            formulas{end} = [formulas{end},sprintf('%d %s',full(model.S(j,i)),model.mets{j})];
        else
            formulas{end} = [formulas{end},' + ',sprintf('%d %s',full(model.S(j,i)),model.mets{j})];
        end
        numproducts = numproducts + 1;
    end
    
end

% add to model variable
model.rxnFormulas = formulas;

Remove quotations from reaction references

In [ ]:
% remove quotations from reaction references
for i = 1:length(model.rxns)
    model.rxnReferences{i} = strrep(strrep(model.rxnReferences{i},"'",""),'"','');
end

Metabolite names

In [ ]:
model.metNames{6588} = model.metNames{6588}(3:end);
model.metNames{7590} = model.metNames{7590}(3:end);
model.metNames{7591} = model.metNames{7591}(3:end);

### IDH1 new reactions

#### IDH1 neomorphic reactions

In [ ]:
model.rxns{end+1} = 'IDH1_R132';
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'akg[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
model.S(strcmp(model.mets,'M00653[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,strcmp(model.geneSymbols,'IDH1')) = 1;
model.rules{end+1} = 'x(1301)';
model.grRules{end+1} = '(3417)';
model.subSystems{end+1} = 'IDH1 neomorphic';
model.rxnNames{end+1} = 'IDH1 neomorphic reaction';
model.rxnKEGGID{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnConfidenceScores(end+1) = 0;
model.rxnReferences{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnNotes{end+1} = '';
model.rxnCOG{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} =  '1 akg[c] + 1 nadph[c] --> 1 M00653[c] + 1 nadp[c]';

#### 2HG c->e reaction

In [ ]:
model.rxns{end+1} = '2HG_c_e';
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'M00653[c]'),end) = -1;
model.S(strcmp(model.mets,'M00653[e]'),end) = 1;
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.subSystems{end+1} = 'IDH1 R132';
model.rxnNames{end+1} = '2HG transport, cytosol to extracellular';
model.rxnKEGGID{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnConfidenceScores(end+1) = 0;
model.rxnReferences{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnNotes{end+1} = '';
model.rxnCOG{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} =  '1 M00653[c] --> 1 M00653[e]';

#### 2HG e-> reaction

In [ ]:
model.rxns{end+1} = '2HG_e_null';
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'M00653[e]'),end) = -1;
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.subSystems{end+1} = 'IDH1 R132';
model.rxnNames{end+1} = '2HG transport, extracellular to null';
model.rxnKEGGID{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnConfidenceScores(end+1) = 0;
model.rxnReferences{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnNotes{end+1} = '';
model.rxnCOG{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} =  '1 M00653[e] --> ';

### Drug modules

#### 5-fluorouracil

Metabolites

In [ ]:
% 5fu[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = '5fu[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C4H3FN2O2';
model.metSmiles{end+1} = 'C1=C(C(=O)NC(=O)N1)F';
model.metNames{end+1} = '5-fluorouracil';
model.metHMDBID{end+1} = 'HMDB0014684';
model.metInChIString{end+1} = 'InChI=1S/C4H3FN2O2/c5-2-1-6-4(9)7-3(2)8/h1H,(H2,6,7,8,9)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '3385';
model.metCHEBIID{end+1} = '46345';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% 5fu[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = '5fu[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C4H3FN2O2';
model.metSmiles{end+1} = 'C1=C(C(=O)NC(=O)N1)F';
model.metNames{end+1} = '5-Fluorouracil';
model.metHMDBID{end+1} = 'HMDB0014684';
model.metInChIString{end+1} = 'InChI=1S/C4H3FN2O2/c5-2-1-6-4(9)7-3(2)8/h1H,(H2,6,7,8,9)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '3385';
model.metCHEBIID{end+1} = '46345';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% fdurd[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fdurd[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C9H11FN2O5';
model.metSmiles{end+1} = 'C1C(C(OC1N2C=C(C(=O)NC2=O)F)CO)O';
model.metNames{end+1} = 'Fluorodeoxyuridine';
model.metHMDBID{end+1} = 'HMDB0014467';
model.metInChIString{end+1} = 'InChI=1S/C9H11FN2O5/c10-4-2-12(9(16)11-8(4)15)7-1-5(14)6(3-13)17-7/h2,5-7,13-14H,1,3H2,(H,11,15,16)/t5-,6+,7+/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '5790';
model.metCHEBIID{end+1} = '60761';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% fdump[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fdump[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C9H10FN2O8P';
model.metSmiles{end+1} = 'C1C(C(OC1(N2C=CC(=O)NC2=O)F)COP(=O)(O)O)O';
model.metNames{end+1} = 'Fluorodeoxyuridine monophosphate';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C9H12FN2O8P/c10-9(12-2-1-7(14)11-8(12)15)3-5(13)6(20-9)4-19-21(16,17)18/h1-2,5-6,13H,3-4H2,(H,11,14,15)(H2,16,17,18)/t5-,6+,9-/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '66697563';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% fdump[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fdump[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C9H10FN2O8P';
model.metSmiles{end+1} = 'C1C(C(OC1(N2C=CC(=O)NC2=O)F)COP(=O)(O)O)O';
model.metNames{end+1} = 'Fluorodeoxyuridine monophosphate';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C9H12FN2O8P/c10-9(12-2-1-7(14)11-8(12)15)3-5(13)6(20-9)4-19-21(16,17)18/h1-2,5-6,13H,3-4H2,(H,11,14,15)(H2,16,17,18)/t5-,6+,9-/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '66697563';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% fdudp[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fdudp[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C9H10FN2O11P2';
model.metSmiles{end+1} = 'C1C(C(OC1(N2C=CC(=O)NC2=O)F)COP(=O)(O)OP(=O)(O)O)O';
model.metNames{end+1} = 'Fluorodeoxyuridine diphosphate';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C9H13FN2O11P2/c10-9(12-2-1-7(14)11-8(12)15)3-5(13)6(22-9)4-21-25(19,20)23-24(16,17)18/h1-2,5-6,13H,3-4H2,(H,19,20)(H,11,14,15)(H2,16,17,18)/t5-,6+,9-/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '87203011';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% furd[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'furd[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C9H11FN2O6';
model.metSmiles{end+1} = 'C1=C(C(=O)NC(=O)N1C2C(C(C(O2)CO)O)O)F';
model.metNames{end+1} = 'Fluorouridine';
model.metHMDBID{end+1} = 'HMDB0060396';
model.metInChIString{end+1} = 'InChI=1S/C9H11FN2O6/c10-3-1-12(9(17)11-7(3)16)8-6(15)5(14)4(2-13)18-8/h1,4-6,8,13-15H,2H2,(H,11,16,17)/t4-,5-,6-,8-/m1/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '9427';
model.metCHEBIID{end+1} = '40154';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% fump[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fump[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C9H10FN2O9P';
model.metSmiles{end+1} = 'C1=CN(C(=O)NC1=O)C2(C(C(C(O2)COP(=O)(O)O)O)O)F';
model.metNames{end+1} = 'Fluorouridine monophosphate';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C9H12FN2O9P/c10-9(12-2-1-5(13)11-8(12)16)7(15)6(14)4(21-9)3-20-22(17,18)19/h1-2,4,6-7,14-15H,3H2,(H,11,13,16)(H2,17,18,19)/t4-,6-,7-,9+/m1/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '67305234';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% fudp[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fudp[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C9H10FN2O12P2';
model.metSmiles{end+1} = 'C1=CN(C(=O)NC1=O)C2(C(C(C(O2)COP(=O)(O)OP(=O)(O)O)O)O)F';
model.metNames{end+1} = 'Fluorouridine diphosphate';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C9H13FN2O12P2/c10-9(12-2-1-5(13)11-8(12)16)7(15)6(14)4(23-9)3-22-26(20,21)24-25(17,18)19/h1-2,4,6-7,14-15H,3H2,(H,20,21)(H,11,13,16)(H2,17,18,19)/t4-,6-,7-,9+/m1/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '87438244';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% dhfu[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'dhfu[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C4H5FN2O2';
model.metSmiles{end+1} = 'C1C(C(=O)NC(=O)N1)F';
model.metNames{end+1} = '5,6-Dihydro-5-fluorouracil';
model.metHMDBID{end+1} = 'HMDB0060402';
model.metInChIString{end+1} = 'InChI=1S/C4H5FN2O2/c5-2-1-6-4(9)7-3(2)8/h2H,1H2,(H2,6,7,8,9)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '121997';
model.metCHEBIID{end+1} = '80624';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% fupa[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fupa[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C4H6FN2O3';
model.metSmiles{end+1} = 'C(C(C(=O)O)F)NC(=O)N';
model.metNames{end+1} = 'Alpha-fluoro-ureidopropionic acid';
model.metHMDBID{end+1} = 'HMDB0060435';
model.metInChIString{end+1} = 'InChI=1S/C4H7FN2O3/c5-2(3(8)9)1-7-4(6)10/h2H,1H2,(H,8,9)(H3,6,7,10)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '151244';
model.metCHEBIID{end+1} = '80625';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';

% fbal[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fbal[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C3H6FNO2';
model.metSmiles{end+1} = 'C(CNF)C(=O)O';
model.metNames{end+1} = 'Fluoro-beta-alanine';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C3H6FNO2/c4-5-2-1-3(6)7/h5H,1-2H2,(H,6,7)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '22665129';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% fbal[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'fbal[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C3H6FNO2';
model.metSmiles{end+1} = 'C(CNF)C(=O)O';
model.metNames{end+1} = 'Fluoro-beta-alanine';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C3H6FNO2/c4-5-2-1-3(6)7/h5H,1-2H2,(H,6,7)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '22665129';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

Reactions

In [ ]:
% 5fu[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'5fu[e]'),end) = -1;
model.rxns{end+1} = 'EX_5fu[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of 5-Fluorouracil';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 5fu[e] <=> ';

% 5fu[e] --> 5fu[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'5fu[e]'),end) = -1;
model.S(strcmp(model.mets,'5fu[c]'),end) = 1;
model.rxns{end+1} = '5fu_transport_e_c';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(666) | x(155)';
model.grRules{end+1} = '(2030) or (10864)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,666) = 1;
model.rxnGeneMat(end,155) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of 5-Fluorouracil from extracellular to cytosol';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 5fu[e] --> 1 5fu[c]';

% 5fu[c] --> 5fu[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'5fu[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'5fu[e]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = '5fu_transport_c_e';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(2996) | x(62) | x(20) | x(2943) | x(3174)';
model.grRules{end+1} = '(8714) or (10257) or (10057) or (85320) or (9429)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,2996) = 1;
model.rxnGeneMat(end,62) = 1;
model.rxnGeneMat(end,20) = 1;
model.rxnGeneMat(end,2943) = 1;
model.rxnGeneMat(end,3174) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of 5-Fluorouracil from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 5fu[c] + 1 atp[c] + 1 h2o[c] --> 1 5fu[e] + 1 adp[c] + 1 pi[c]';

% 5fu[c] <=> fdurd[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'5fu[c]'),end) = -1;
model.S(strcmp(model.mets,'2dr1p[c]'),end) = -1;
model.S(strcmp(model.mets,'fdurd[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = '5fu_fdurd';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(630)';
model.grRules{end+1} = '(1890)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,630) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = '5-Fluorouracil to Fluorodeoxyuridine';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.4.2.2 | 2.4.2.4';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 5fu[c] + 1 2dr1p[c] <=> 1 fdurd[c] + 1 pi[c]';

% fdurd[c] --> fdump[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'fdurd[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'fdump[c]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = 'fdurd_fdump';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(2601)';
model.grRules{end+1} = '(7083)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,2601) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Fluorodeoxyuridine to Fluorodeoxyuridine monophosphate';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.7.1.21 | 2.7.1.145';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 fdurd[c] + 1 atp[c] --> 1 fdump[c] + 1 adp[c] + 1 h[c]';

% fdudp[c] <=> fdump[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'fdump[c]'),end) = 1;
model.S(strcmp(model.mets,'atp[c]'),end) = 1;
model.S(strcmp(model.mets,'fdudp[c]'),end) = -1;
model.S(strcmp(model.mets,'adp[c]'),end) = -1;
model.rxns{end+1} = 'fdudp_fdump';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Fluorodeoxyuridine diphosphate to Fluorodeoxyuridine monophosphate';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.7.4.4 | 2.7.4.9';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 fdudp[c] + 1 adp[c] <=> 1 fdump[c] + 1 atp[c]';

% 5fu[c] <=> furd[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'5fu[c]'),end) = -1;
model.S(strcmp(model.mets,'r1p[c]'),end) = -1;
model.S(strcmp(model.mets,'furd[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = '5fu_furd';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(2661) | x(478)';
model.grRules{end+1} = '(7378) or (151531)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,2661) = 1;
model.rxnGeneMat(end,478) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = '5-Fluorouracil to Fluorouridine';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.4.2.2 | 2.4.2.3';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 5fu[c] + 1 r1p[c] <=> 1 furd[c] + 1 pi[c]';

% furd[c] --> fump[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'furd[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'fump[c]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = 'furd_fump';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(2839) | x(2658) | x(1987)';
model.grRules{end+1} = '(83549) or (7371) or (54963)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,2839) = 1;
model.rxnGeneMat(end,2658) = 1;
model.rxnGeneMat(end,1987) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Fluorouridine to Fluorouridine monophosphate';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.7.1.48 | 2.7.1.213';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 furd[c] + 1 atp[c] --> 1 fump[c] + 1 adp[c] + 1 h[c]';

% 5fu[c] <=> fump[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'5fu[c]'),end) = -1;
model.S(strcmp(model.mets,'prpp[c]'),end) = -1;
model.S(strcmp(model.mets,'fump[c]'),end) = 1;
model.S(strcmp(model.mets,'ppi[c]'),end) = 1;
model.rxns{end+1} = '5fu_fump';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1960) | x(2659)';
model.grRules{end+1} = '(5471) or (7372)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1960) = 1;
model.rxnGeneMat(end,2659) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = '5-Fluorouracil to Fluorouridine monophosphate';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.4.2.9';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 5fu[c] + 1 prpp[c] <=> 1 fump[c] + 1 ppi[c]';

% fump[c] <=> fudp[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'fump[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'fudp[c]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.rxns{end+1} = 'fump_fudp';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Fluorouridine monophosphate to Fluorouridine diphosphate';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.7.4.4 | 2.7.4.14 | 2.7.4.22';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 fump[c] + 1 atp[c] <=> 1 fudp[c] + 1 adp[c]';

% fudp[c] --> fdudp[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'fudp[c]'),end) = -1;
model.S(strcmp(model.mets,'trdrd[c]'),end) = -1;
model.S(strcmp(model.mets,'fdudp[c]'),end) = 1;
model.S(strcmp(model.mets,'trdox[c]'),end) = 1;
model.S(strcmp(model.mets,'h2o[c]'),end) = 1;
model.rxns{end+1} = 'fudp_fdudp';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(2348) & (x(2349) | x(1635))';
model.grRules{end+1} = '(6240) and ((6241) or (50484))';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,2348) = 1;
model.rxnGeneMat(end,2349) = 1;
model.rxnGeneMat(end,1635) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Fluorouridine diphosphate to Fluorodeoxyuridine diphosphate';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.17.4.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 fudp[c] + 1 trdrd[c] --> 1 fdudp[c] + 1 trdox[c] + 1 h2o[c]';

% 5fu[c] --> dhfu[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'5fu[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
model.S(strcmp(model.mets,'h[c]'),end) = -1;
model.S(strcmp(model.mets,'dhfu[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
model.rxns{end+1} = '5fu_dhfu';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(610)';
model.grRules{end+1} = '(1806)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,610) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = '5-Fluorouracil to 5,6-Dihydro-5-fluorouracil';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.3.1.2';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 5fu[c] + 1 nadph[c] + 1 h[c] --> 1 dhfu[c] + 1 nadp[c]';

% dhfu[c] --> fupa[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'dhfu[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'fupa[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = 'dhfu_fupa';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(611) | x(612) | x(613)';
model.grRules{end+1} = '(1807) or (1808) or (1809)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,611) = 1;
model.rxnGeneMat(end,612) = 1;
model.rxnGeneMat(end,613) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = '5,6-Dihydro-5-fluorouracil to alpha-fluoro-ureidopropionic acid';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '3.5.2.2';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 dhfu[c] + 1 h2o[c] --> 1 fupa[c] + 1 h[c]';

% fupa[c] --> fbal[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'fupa[c]'),end) = -1;
model.S(strcmp(model.mets,'h[c]'),end) = -2;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'fbal[c]'),end) = 1;
model.S(strcmp(model.mets,'co2[c]'),end) = 1;
model.S(strcmp(model.mets,'nh4[c]'),end) = 1;
model.rxns{end+1} = 'fupa_fbal';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1793)';
model.grRules{end+1} = '(51733)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1793) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Alpha-fluoro-ureidopropionic acid to fluoro-beta-alanine';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '3.5.1.6';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 fupa[c] + 2 h[c] + 1 h2o[c] --> 1 fbal[c] + 1 co2[c] + 1 nh4[c]';

% fbal[c] <=> fbal[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'fbal[e]'),end) = 1;
model.S(strcmp(model.mets,'fbal[c]'),end) = -1;
model.rxns{end+1} = 'fbal_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Fluoro-beta-alanine from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 fbal[c] <=> 1 fbal[e]';

% fbal[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'fbal[e]'),end) = -1;
model.rxns{end+1} = 'EX_fbal[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Fluoro-beta-alanine';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 fbal[e] <=> ';

% artificial fdump[c] sink
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'fdump[c]'),end) = -1;
model.rxns{end+1} = 'SINK_fdump[c]';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Artificial sink on Fluorodeoxyuridine monophosphate';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE 5-FLUOROURACIL';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 fdump[c] --> ';

#### Doxorubicin

Metabolites

In [151]:
% doxQ[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'doxQ[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C27H29NO11';
model.metSmiles{end+1} = 'CC1C(C(CC(O1)OC2CC(CC3=C2C(=C4C(=C3O)C(=O)C5=C(C4=O)C(=CC=C5)OC)O)(C(=O)CO)O)N)O';
model.metNames{end+1} = 'Doxorubicin';
model.metHMDBID{end+1} = 'HMDB0015132';
model.metInChIString{end+1} = 'InChI=1S/C27H29NO11/c1-10-22(31)13(28)6-17(38-10)39-15-8-27(36,16(30)9-29)7-12-19(15)26(35)21-20(24(12)33)23(32)11-4-3-5-14(37-2)18(11)25(21)34/h3-5,10,13,15,17,22,29,31,33,35-36H,6-9,28H2,1-2H3/t10-,13-,15-,17-,22+,27-/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '31703';
model.metCHEBIID{end+1} = '28748';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% doxQ[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'doxQ[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C27H29NO11';
model.metSmiles{end+1} = 'CC1C(C(CC(O1)OC2CC(CC3=C2C(=C4C(=C3O)C(=O)C5=C(C4=O)C(=CC=C5)OC)O)(C(=O)CO)O)N)O';
model.metNames{end+1} = 'Doxorubicin';
model.metHMDBID{end+1} = 'HMDB0015132';
model.metInChIString{end+1} = 'InChI=1S/C27H29NO11/c1-10-22(31)13(28)6-17(38-10)39-15-8-27(36,16(30)9-29)7-12-19(15)26(35)21-20(24(12)33)23(32)11-4-3-5-14(37-2)18(11)25(21)34/h3-5,10,13,15,17,22,29,31,33,35-36H,6-9,28H2,1-2H3/t10-,13-,15-,17-,22+,27-/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '31703';
model.metCHEBIID{end+1} = '28748';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% doxSQ[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'doxSQ[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C27H30NO11';
model.metSmiles{end+1} = 'CC1C(C(CC(O1)OC2CC(CC3=C2C(=C4C(C5=C(C=CC=C5OC)C(C4=C3O)O)O)O)(C(=O)CO)O)N)O';
model.metNames{end+1} = 'Doxorubicin semiquinone';
model.metHMDBID{end+1} = 'HMDB0060822';
model.metInChIString{end+1} = 'InChI=1S/C27H33NO11/c1-10-22(31)13(28)6-17(38-10)39-15-8-27(36,16(30)9-29)7-12-19(15)26(35)21-20(24(12)33)23(32)11-4-3-5-14(37-2)18(11)25(21)34/h3-5,10,13,15,17,22-23,25,29,31-36H,6-9,28H2,1-2H3';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '131769967';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% doxHQ[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'doxHQ[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C27H31NO11';
model.metSmiles{end+1} = 'CC1C(C(CC(O1)OC2CC(CC3=C2C(=C4C(=C3O)C(=O)C5=C(C4=O)C(=CC=C5)OC)O)(C(CO)O)O)N)O';
model.metNames{end+1} = 'Doxorubicinol';
model.metHMDBID{end+1} = 'HMDB0041884';
model.metInChIString{end+1} = 'InChI=1S/C27H31NO11/c1-10-22(31)13(28)6-17(38-10)39-15-8-27(36,16(30)9-29)7-12-19(15)26(35)21-20(24(12)33)23(32)11-4-3-5-14(37-2)18(11)25(21)34/h3-5,10,13,15-17,22,29-31,33,35-36H,6-9,28H2,1-2H3/t10-,13-,15-,16-,17-,22+,27-/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '83970';
model.metCHEBIID{end+1} = '133817';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% doxHQ[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'doxHQ[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C27H31NO11';
model.metSmiles{end+1} = 'CC1C(C(CC(O1)OC2CC(CC3=C2C(=C4C(=C3O)C(=O)C5=C(C4=O)C(=CC=C5)OC)O)(C(CO)O)O)N)O';
model.metNames{end+1} = 'Doxorubicinol';
model.metHMDBID{end+1} = 'HMDB0041884';
model.metInChIString{end+1} = 'InChI=1S/C27H31NO11/c1-10-22(31)13(28)6-17(38-10)39-15-8-27(36,16(30)9-29)7-12-19(15)26(35)21-20(24(12)33)23(32)11-4-3-5-14(37-2)18(11)25(21)34/h3-5,10,13,15-17,22,29-31,33,35-36H,6-9,28H2,1-2H3/t10-,13-,15-,16-,17-,22+,27-/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '83970';
model.metCHEBIID{end+1} = '133817';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% doxdeoxy[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'doxdeoxy[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C21H18O8';
model.metSmiles{end+1} = 'COC1=CC=CC2=C1C(=O)C3=C(C4=C(CC(CC4)(C(=O)CO)O)C(=C3C2=O)O)O';
model.metNames{end+1} = '7-Deoxydoxorubicinone';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C21H18O8/c1-29-12-4-2-3-10-14(12)20(27)16-15(18(10)25)19(26)11-7-21(28,13(23)8-22)6-5-9(11)17(16)24/h2-4,22,24,26,28H,5-8H2,1H3/t21-/m1/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '83958';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% doxdeoxy[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'doxdeoxy[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C21H18O8';
model.metSmiles{end+1} = 'COC1=CC=CC2=C1C(=O)C3=C(C4=C(CC(CC4)(C(=O)CO)O)C(=C3C2=O)O)O';
model.metNames{end+1} = '7-Deoxydoxorubicinone';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C21H18O8/c1-29-12-4-2-3-10-14(12)20(27)16-15(18(10)25)19(26)11-7-21(28,13(23)8-22)6-5-9(11)17(16)24/h2-4,22,24,26,28H,5-8H2,1H3/t21-/m1/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '83958';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% daun[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'daun[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C6H13NO3';
model.metSmiles{end+1} = 'CC(C(C(CC=O)N)O)O';
model.metNames{end+1} = 'Daunosamine';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C6H13NO3/c1-4(9)6(10)5(7)2-3-8/h3-6,9-10H,2,7H2,1H3/t4-,5-,6+/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '160128';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% daun[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'daun[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C6H13NO3';
model.metSmiles{end+1} = 'CC(C(C(CC=O)N)O)O';
model.metNames{end+1} = 'Daunosamine';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/C6H13NO3/c1-4(9)6(10)5(7)2-3-8/h3-6,9-10H,2,7H2,1H3/t4-,5-,6+/m0/s1';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '160128';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

Reactions

In [152]:
% % % import metabolites

% doxQ[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxQ[e]'),end) = -1;
model.rxns{end+1} = 'EX_doxQ[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Doxorubicin';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxQ[e] <=> ';

% doxHQ[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxHQ[e]'),end) = -1;
model.rxns{end+1} = 'EX_doxHQ[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Doxorubicinol';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxHQ[e] <=> ';

% doxdeoxy[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxdeoxy[e]'),end) = -1;
model.rxns{end+1} = 'EX_doxdeoxy[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of 7-Deoxydoxorubicinone';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxdeoxy[e] <=> ';

% daun[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'daun[e]'),end) = -1;
model.rxns{end+1} = 'EX_daun[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Daunosamine';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 daun[e] <=> ';

% % % transport

% doxQ[e] --> doxQ[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxQ[e]'),end) = -1;
model.S(strcmp(model.mets,'doxQ[c]'),end) = 1;
model.rxns{end+1} = 'doxQ_transport_e_c';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(2949)';
model.grRules{end+1} = '(85413)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,2949) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Doxorubicin from extracellular to cytosol';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxQ[e] --> 1 doxQ[c]';

% doxQ[c] --> doxQ[e], no ATP
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxQ[c]'),end) = -1;
model.S(strcmp(model.mets,'doxQ[e]'),end) = 1;
model.rxns{end+1} = 'doxQ_transport_c_e_noATP';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(167)';
model.grRules{end+1} = '(10928)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,167) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Doxorubicin from cytosol to extracellular, no ATP';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxQ[c] --> 1 doxQ[e]';

% doxQ[c] --> doxQ[e], with ATP
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxQ[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'doxQ[e]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = 'doxQ_transport_c_e_withATP';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1823) | x(1499) | x(331) | x(62) | x(3174)';
model.grRules{end+1} = '(5243) or (4363) or (1244) or (10257) or (9429)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1823) = 1;
model.rxnGeneMat(end,1499) = 1;
model.rxnGeneMat(end,331) = 1;
model.rxnGeneMat(end,62) = 1;
model.rxnGeneMat(end,3174) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Doxorubicin from cytosol to extracellular, with ATP';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxQ[c] + 1 atp[c] + 1 h2o[c] --> 1 doxQ[e] + 1 adp[c] + 1 pi[c]';

% doxHQ[c] <=> doxHQ[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxHQ[e]'),end) = 1;
model.S(strcmp(model.mets,'doxHQ[c]'),end) = -1;
model.rxns{end+1} = 'doxHQ_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Doxorubicinol from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxHQ[c] <=> 1 doxHQ[e]';

% doxdeoxy[c] <=> doxdeoxy[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxdeoxy[e]'),end) = 1;
model.S(strcmp(model.mets,'doxdeoxy[c]'),end) = -1;
model.rxns{end+1} = 'doxdeoxy_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of 7-Deoxydoxorubicinone from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxdeoxy[c] <=> 1 doxdeoxy[e]';

% daun[c] <=> daun[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'daun[e]'),end) = 1;
model.S(strcmp(model.mets,'daun[c]'),end) = -1;
model.rxns{end+1} = 'daun_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Daunosamine from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 daun[c] <=> 1 daun[e]';

% % % doxQ and doxSQ

% doxQ[c] --> doxSQ[c], NOS3
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxQ[c]'),end) = -1;
model.S(strcmp(model.mets,'o2[c]'),end) = -2;
model.S(strcmp(model.mets,'arg_L[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -1.5;
model.S(strcmp(model.mets,'h[c]'),end) = -1.5;
model.S(strcmp(model.mets,'doxSQ[c]'),end) = 1;
model.S(strcmp(model.mets,'h2o[c]'),end) = 2;
model.S(strcmp(model.mets,'no[c]'),end) = 1;
model.S(strcmp(model.mets,'citr_L[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 1.5;
model.rxns{end+1} = 'doxQ_doxSQ_NOS3';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1598)';
model.grRules{end+1} = '(4846)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1598) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Doxorubicin to doxorubicin semiquinone, NOS3';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.14.13.39';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxQ[c] + 2 o2[c] + 1 arg_L[c] + 1.5 nadph[c] + 1.5 h[c] --> 1 doxSQ[c] + 2 h2o[c] + 1 no[c] + 1 citr_L[c] + 1.5 nadp[c]';

% doxQ[c] --> doxSQ[c], POR
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxQ[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -0.5;
model.S(strcmp(model.mets,'h[c]'),end) = -0.5;
model.S(strcmp(model.mets,'doxSQ[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 0.5;
model.rxns{end+1} = 'doxQ_doxSQ_POR';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1933)';
model.grRules{end+1} = '(5447)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1933) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Doxorubicin to doxorubicin semiquinone, POR';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.6.2.4';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxQ[c] + 0.5 nadph[c] + 0.5 h[c] --> 1 doxSQ[c] + 0.5 nadp[c]';

% doxSQ[c] --> doxQ
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxSQ[c]'),end) = -1;
model.S(strcmp(model.mets,'o2[c]'),end) = -1;
model.S(strcmp(model.mets,'doxQ[c]'),end) = 1;
model.S(strcmp(model.mets,'o2s[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = 'doxSQ_doxQ';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Doxorubicin semiquinone to doxorubicin';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxSQ[c] + 1 o2[c] --> 1 doxQ[c] + 1 o2s[c] + 1 h[c]';

% % % doxQ metabolism

% doxQ[c] --> doxHQ[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxQ[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
model.S(strcmp(model.mets,'h[c]'),end) = -1;
model.S(strcmp(model.mets,'doxHQ[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
model.rxns{end+1} = 'doxQ_doxHQ';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(2999) | x(3004) | x(74) | x(2978)';
model.grRules{end+1} = '(873) or (874) or (10327) or (8644)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,2999) = 1;
model.rxnGeneMat(end,3004) = 1;
model.rxnGeneMat(end,74) = 1;
model.rxnGeneMat(end,2978) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Doxorubicin to doxorubicinol';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.1.1.2 | 1.1.1.184';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxQ[c] + 1 nadph[c] + 1 h[c] --> 1 doxHQ[c] + 1 nadp[c]';

% doxQ[c] --> doxdeoxy[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'doxQ[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -0.5;
model.S(strcmp(model.mets,'h[c]'),end) = -1.5;
model.S(strcmp(model.mets,'doxdeoxy[c]'),end) = 1;
model.S(strcmp(model.mets,'daun[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 0.5;
model.rxns{end+1} = 'doxQ_doxdeoxy';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1933)';
model.grRules{end+1} = '(5447)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1933) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Doxorubicin to 7-deoxydoxorubicinone';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.6.2.4';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE DOXORUBICIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 doxQ[c] + 0.5 nadph[c] + 1.5 h[c] --> 1 doxHQ[c] + 1 daun[c] + 0.5 nadp[c]';

#### Cisplatin

Metabolites

In [153]:
% cis[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cis[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'Cl2H6N2Pt';
model.metSmiles{end+1} = 'N.N.Cl[Pt]Cl';
model.metNames{end+1} = 'Cisplatin';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/2ClH.2H3N.Pt/h2*1H;2*1H3;/q;;;;+2/p-2';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '5702198';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cis[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cis[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'Cl2H6N2Pt';
model.metSmiles{end+1} = 'N.N.Cl[Pt]Cl';
model.metNames{end+1} = 'Cisplatin';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/2ClH.2H3N.Pt/h2*1H;2*1H3;/q;;;;+2/p-2';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '5702198';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cis[m]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cis[m]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'Cl2H6N2Pt';
model.metSmiles{end+1} = 'N.N.Cl[Pt]Cl';
model.metNames{end+1} = 'Cisplatin';m
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = 'InChI=1S/2ClH.2H3N.Pt/h2*1H;2*1H3;/q;;;;+2/p-2';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '5702198';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cis-mtDNA[m]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cis-mtDNA[m]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = '';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Cisplatin-mitochondrial DNA';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cisGSH[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cisGSH[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C10Cl2H21N5O6PtS';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Cisplatin-GSH';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cisGSH[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cisGSH[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C10Cl2H21N5O6PtS';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Cisplatin-GSH';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cisMT[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cisMT[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = '';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Cisplatin-metallothionein';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cisMT[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cisMT[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = '';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Cisplatin-metallothionein';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

Reactions

In [154]:
% % % import metabolites

% cis[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cis[e]'),end) = -1;
model.rxns{end+1} = 'EX_cis[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Cisplatin';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cis[e] <=> ';

% cisGSH[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cisGSH[e]'),end) = -1;
model.rxns{end+1} = 'EX_cisGSH[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Cisplatin-GSH';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cisGSH[e] <=> ';

% cisMT[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cisMT[e]'),end) = -1;
model.rxns{end+1} = 'EX_cisMT[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Cisplatin-metallothionein';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cisMT[e] <=> ';

% % % transport

% cis[e] --> cis[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cis[e]'),end) = -1;
model.S(strcmp(model.mets,'cis[c]'),end) = 1;
model.rxns{end+1} = 'cis_transport_e_c';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(375)';
model.grRules{end+1} = '(1317)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,375) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Cisplatin from extracellular to cytosol';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cis[e] --> 1 cis[c]';

% cis[c] --> cis[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cis[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'cis[e]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = 'cis_transport_c_e';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(3174) | x(3269) | x(3270)';
model.grRules{end+1} = '(9429) or (538) or (540)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,3174) = 1;
model.rxnGeneMat(:,(end+1):(end+2)) = zeros(length(model.rxns),2);
model.rxnGeneMat(end,3269) = 1;
model.rxnGeneMat(end,3270) = 1;
model.genes{end+1} = '538';
model.genes{end+1} = '540';
model.geneSymbols{end+1} = 'ATP7A';
model.geneSymbols{end+1} = 'ATP7B';
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Cisplatin from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cis[c] + 1 atp[c] + 1 h2o[c] --> 1 cis[e] + 1 adp[c] + 1 pi[c]';

% cisGSH[c] --> cisGSH[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cisGSH[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'cisGSH[e]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = 'cisGSH_transport_c_e';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(331)';
model.grRules{end+1} = '(1244)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,331) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Cisplatin-GSH from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cisGSH[c] + 1 atp[c] + 1 h2o[c] --> 1 cisGSH[e] + 1 adp[c] + 1 pi[c]';

% cisMT[c] <=> cisMT[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cisMT[e]'),end) = 1;
model.S(strcmp(model.mets,'cisMT[c]'),end) = -1;
model.rxns{end+1} = 'cisMT_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Cisplatin-metallothionein from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cisMT[c] <=> 1 cisMT[e]';

% cis[c] --> cis[m]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cis[c]'),end) = -1;
model.S(strcmp(model.mets,'cis[m]'),end) = 1;
model.rxns{end+1} = 'cis_transport_c_m';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(3271)';
model.grRules{end+1} = '(10063)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(:,end+1) = zeros(length(model.rxns),1);
model.rxnGeneMat(end,3271) = 1;
model.genes{end+1} = '10063';
model.geneSymbols{end+1} = 'COX17';
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Cisplatin from cytosol to mitochondrion';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.9.3.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cis[c] --> 1 cis[m]';

% % % cisplatin mechanism

% cis[m] --> cis-mtDNA[m]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cis[m]'),end) = -1;
model.S(strcmp(model.mets,'o2[m]'),end) = -0.5;
model.S(strcmp(model.mets,'cis-mtDNA[m]'),end) = 1;
model.S(strcmp(model.mets,'o2s[m]'),end) = 0.5;
model.rxns{end+1} = 'cis_cis-mtDNA';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1519) | x(1520) | x(1521) | x(1522) | x(1523) | x(1524) | x(1525) | x(1516)';
model.grRules{end+1} = '(4535) or (4536) or (4537) or (4538) or (4539) or (4540) or (4541) or (4519)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1519) = 1;
model.rxnGeneMat(end,1520) = 1;
model.rxnGeneMat(end,1521) = 1;
model.rxnGeneMat(end,1522) = 1;
model.rxnGeneMat(end,1523) = 1;
model.rxnGeneMat(end,1524) = 1;
model.rxnGeneMat(end,1525) = 1;
model.rxnGeneMat(end,1516) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Cisplatin to cisplatin-mtDNA';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.9.3.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cis[m] + 0.5 o2[m] --> 1 cis-mtDNA[m] + 0.5 o2s[m]';

% cisplatin clearance

% cis[c] --> cisGSH[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cis[c]]'),end) = -1;
model.S(strcmp(model.mets,'gthrd[c]'),end) = -1;
model.S(strcmp(model.mets,'cisGSH[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = 'cis_cisGSH';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1179) | x(3176) | x(1184) | x(1185)';
model.grRules{end+1} = '(2944) or (9446) or (2950) or (2952)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1179) = 1;
model.rxnGeneMat(end,3176) = 1;
model.rxnGeneMat(end,1184) = 1;
model.rxnGeneMat(end,1185) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Cisplatin to cisplatin-GSH';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.5.1.18';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cis[c] + 1 gthrd[c] --> 1 cisGSH[c] + 1 h[c]';

% cis[c] --> cisMT[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cis[c]]'),end) = -2;
model.S(strcmp(model.mets,'gthrd[c]'),end) = -2;
model.S(strcmp(model.mets,'cisMT[c]'),end) = 1;
model.S(strcmp(model.mets,'gthox[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 2;
model.rxns{end+1} = 'cis_cisMT';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(3272) | x(3273) | x(3274) | x(3275)';
model.grRules{end+1} = '1';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(:,(end+1):(end+4)) = zeros(length(model.rxns),4);
model.rxnGeneMat(end,3272) = 1;
model.rxnGeneMat(end,3273) = 1;
model.rxnGeneMat(end,3274) = 1;
model.rxnGeneMat(end,3275) = 1;
model.genes{end+1} = '4489';
model.genes{end+1} = '4502';
model.genes{end+1} = '4504';
model.genes{end+1} = '84560';
model.geneSymbols{end+1} = 'MT1A';
model.geneSymbols{end+1} = 'MT2A';
model.geneSymbols{end+1} = 'MT3';
model.geneSymbols{end+1} = 'MT4';
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Cisplatin to cisplatin-metallothionein';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.8.5.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '2 cis[c] + 2 gthrd[c] --> 1 cisMT[c] + 1 gthox[c] + 2 h[c]';

% % % sinks

% artificial cis-mtDNA[m] sink
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cis-mtDNA[m]'),end) = -1;
model.rxns{end+1} = 'SINK_cis-mtDNA[m]';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Artificial sink on Cisplatin-mitochondrial DNA';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CISPLATIN';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cis-mtDNA[m] --> ';

#### Cyclophosphamide

Metabolites

In [155]:
% cpa[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cpa[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H15Cl2N2O2P';
model.metSmiles{end+1} = 'C1CNP(=O)(OC1)N(CCCl)CCCl';
model.metNames{end+1} = 'Cyclophosphamide';
model.metHMDBID{end+1} = 'HMDB0014672';
model.metInChIString{end+1} = 'InChI=1S/C7H15Cl2N2O2P/c8-2-5-11(6-3-9)14(12)10-4-1-7-13-14/h1-7H2,(H,10,12)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '2907';
model.metCHEBIID{end+1} = '4027';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cpa[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cpa[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H15Cl2N2O2P';
model.metSmiles{end+1} = 'C1CNP(=O)(OC1)N(CCCl)CCCl';
model.metNames{end+1} = 'Cyclophosphamide';
model.metHMDBID{end+1} = 'HMDB0014672';
model.metInChIString{end+1} = 'InChI=1S/C7H15Cl2N2O2P/c8-2-5-11(6-3-9)14(12)10-4-1-7-13-14/h1-7H2,(H,10,12)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '2907';
model.metCHEBIID{end+1} = '4027';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% dce-cpa[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'dce-cpa[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C5H12ClN2O2P';
model.metSmiles{end+1} = 'C1CNP(=O)(OC1)NCCCl';
model.metNames{end+1} = 'Dechloroethylcyclophosphamide';
model.metHMDBID{end+1} = 'HMDB0013858';
model.metInChIString{end+1} = 'InChI=1S/C5H12ClN2O2P/c6-2-4-8-11(9)7-3-1-5-10-11/h1-5H2,(H2,7,8,9)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '114861';
model.metCHEBIID{end+1} = '80558';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% dce-cpa[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'dce-cpa[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C5H12ClN2O2P';
model.metSmiles{end+1} = 'C1CNP(=O)(OC1)NCCCl';
model.metNames{end+1} = 'Dechloroethylcyclophosphamide';
model.metHMDBID{end+1} = 'HMDB0013858';
model.metInChIString{end+1} = 'InChI=1S/C5H12ClN2O2P/c6-2-4-8-11(9)7-3-1-5-10-11/h1-5H2,(H2,7,8,9)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '114861';
model.metCHEBIID{end+1} = '80558';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% caa[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'caa[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C2H3ClO';
model.metSmiles{end+1} = 'C(C=O)Cl';
model.metNames{end+1} = 'Chloroacetaldehyde';
model.metHMDBID{end+1} = 'HMDB0013860';
model.metInChIString{end+1} = 'InChI=1S/C2H3ClO/c3-1-2-4/h2H,1H2';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '33';
model.metCHEBIID{end+1} = '27871';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% caa[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'caa[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C2H3ClO';
model.metSmiles{end+1} = 'C(C=O)Cl';
model.metNames{end+1} = 'Chloroacetaldehyde';
model.metHMDBID{end+1} = 'HMDB0013860';
model.metInChIString{end+1} = 'InChI=1S/C2H3ClO/c3-1-2-4/h2H,1H2';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '33';
model.metCHEBIID{end+1} = '27871';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% 4oh-cpa[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = '4oh-cpa[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H15Cl2N2O3P';
model.metSmiles{end+1} = 'C1COP(=O)(NC1O)N(CCCl)CCCl';
model.metNames{end+1} = '4-Hydroxycyclophosphamide';
model.metHMDBID{end+1} = 'HMDB0013856';
model.metInChIString{end+1} = 'InChI=1S/C7H15Cl2N2O3P/c8-2-4-11(5-3-9)15(13)10-7(12)1-6-14-15/h7,12H,1-6H2,(H,10,13)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '99735';
model.metCHEBIID{end+1} = '1864';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% 4k-cpa[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = '4k-cpa[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H13Cl2N2O3P';
model.metSmiles{end+1} = 'C1COP(=O)(NC1=O)N(CCCl)CCCl';
model.metNames{end+1} = '4-Ketocyclophosphamide';
model.metHMDBID{end+1} = 'HMDB0060686';
model.metInChIString{end+1} = 'InChI=1S/C7H13Cl2N2O3P/c8-2-4-11(5-3-9)15(13)10-7(12)1-6-14-15/h1-6H2,(H,10,12,13)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '33676';
model.metCHEBIID{end+1} = '1887';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% 4k-cpa[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = '4k-cpa[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H13Cl2N2O3P';
model.metSmiles{end+1} = 'C1COP(=O)(NC1=O)N(CCCl)CCCl';
model.metNames{end+1} = '4-Ketocyclophosphamide';
model.metHMDBID{end+1} = 'HMDB0060686';
model.metInChIString{end+1} = 'InChI=1S/C7H13Cl2N2O3P/c8-2-4-11(5-3-9)15(13)10-7(12)1-6-14-15/h1-6H2,(H,10,12,13)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '33676';
model.metCHEBIID{end+1} = '1887';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% aldph[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'aldph[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H15Cl2N2O3P';
model.metSmiles{end+1} = 'C(COP(=O)(N)N(CCCl)CCCl)C=O';
model.metNames{end+1} = 'Aldophosphamide';
model.metHMDBID{end+1} = 'HMDB0060433';
model.metInChIString{end+1} = 'InChI=1S/C7H15Cl2N2O3P/c8-2-4-11(5-3-9)15(10,13)14-7-1-6-12/h6H,1-5,7H2,(H2,10,13)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '107744';
model.metCHEBIID{end+1} = '2560';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% alcph[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'alcph[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H17Cl2N2O3P';
model.metSmiles{end+1} = 'C(CO)COP(=O)(N)N(CCCl)CCCl';
model.metNames{end+1} = 'Alcophosphamide';
model.metHMDBID{end+1} = 'HMDB0060432';
model.metInChIString{end+1} = 'InChI=1S/C7H17Cl2N2O3P/c8-2-4-11(5-3-9)15(10,13)14-7-1-6-12/h12H,1-7H2,(H2,10,13)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '98612';
model.metCHEBIID{end+1} = '80559';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% alcph[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'alcph[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H17Cl2N2O3P';
model.metSmiles{end+1} = 'C(CO)COP(=O)(N)N(CCCl)CCCl';
model.metNames{end+1} = 'Alcophosphamide';
model.metHMDBID{end+1} = 'HMDB0060432';
model.metInChIString{end+1} = 'InChI=1S/C7H17Cl2N2O3P/c8-2-4-11(5-3-9)15(10,13)14-7-1-6-12/h12H,1-7H2,(H2,10,13)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '98612';
model.metCHEBIID{end+1} = '80559';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cph[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cph[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H15Cl2N2O4P';
model.metSmiles{end+1} = 'C(COP(=O)(N)N(CCCl)CCCl)C(=O)O';
model.metNames{end+1} = 'Carboxyphosphamide';
model.metHMDBID{end+1} = 'HMDB0060449';
model.metInChIString{end+1} = 'InChI=1S/C7H15Cl2N2O4P/c8-2-4-11(5-3-9)16(10,14)15-6-1-7(12)13/h1-6H2,(H2,10,14)(H,12,13)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '31515';
model.metCHEBIID{end+1} = '3410';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% cph[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'cph[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C7H15Cl2N2O4P';
model.metSmiles{end+1} = 'C(COP(=O)(N)N(CCCl)CCCl)C(=O)O';
model.metNames{end+1} = 'Carboxyphosphamide';
model.metHMDBID{end+1} = 'HMDB0060449';
model.metInChIString{end+1} = 'InChI=1S/C7H15Cl2N2O4P/c8-2-4-11(5-3-9)16(10,14)15-6-1-7(12)13/h1-6H2,(H2,10,14)(H,12,13)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '31515';
model.metCHEBIID{end+1} = '3410';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% phmust[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'phmust[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C4H11Cl2N2O2P';
model.metSmiles{end+1} = 'C(CCl)N(CCCl)P(=O)(N)O';
model.metNames{end+1} = 'Phosphoramide mustard';
model.metHMDBID{end+1} = 'HMDB0060689';
model.metInChIString{end+1} = 'InChI=1S/C4H11Cl2N2O2P/c5-1-3-8(4-2-6)11(7,9)10/h1-4H2,(H3,7,9,10)';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '96356';
model.metCHEBIID{end+1} = '8163';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% acr[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'acr[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C3H4O';
model.metSmiles{end+1} = 'C=CC=O';
model.metNames{end+1} = 'Acrolein';
model.metHMDBID{end+1} = 'HMDB0041822';
model.metInChIString{end+1} = 'InChI=1S/C3H4O/c1-2-3-4/h2-3H,1H2';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '7847';
model.metCHEBIID{end+1} = '15368';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% phmust-GSH[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'phmust-GSH[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C14H26Cl2N5O8PS';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Phosphoramide mustard-GSH';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% phmust-GSH[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'phmust-GSH[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C14H26Cl2N5O8PS';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Phosphoramide mustard-GSH';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% acr-GSH[e]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'acr-GSH[e]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C13H19N3O7S';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Acrolein-GSH';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% acr-GSH[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'acr-GSH[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'C13H19N3O7S';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Acrolein-GSH';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

Reactions

In [156]:
% % % import metabolites

% cpa[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cpa[e]'),end) = -1;
model.rxns{end+1} = 'EX_cpa[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Cyclophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cpa[e] <=> ';

% dce-cpa[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'dce-cpa[e]'),end) = -1;
model.rxns{end+1} = 'EX_dce-cpa[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Dechloroethylcyclophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 dce-cpa[e] <=> ';

% caa[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'caa[e]'),end) = -1;
model.rxns{end+1} = 'EX_caa[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Chloroacetaldehyde';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 caa[e] <=> ';

% 4k-cpa[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'4k-cpa[e]'),end) = -1;
model.rxns{end+1} = 'EX_4k-cpa[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of 4-Ketocyclophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 4k-cpa[e] <=> ';

% alcph[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'alcph[e]'),end) = -1;
model.rxns{end+1} = 'EX_alcph[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Alcophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 alcph[e] <=> ';

% cph[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cph[e]'),end) = -1;
model.rxns{end+1} = 'EX_cph[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Carboxyphosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cph[e] <=> ';

% phmust-GSH[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'phmust-GSH[e]'),end) = -1;
model.rxns{end+1} = 'EX_phmust-GSH[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Phosphoramide mustard-GSH';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 phmust-GSH[e] <=> ';

% acr-GSH[e] <=>
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'acr-GSH[e]'),end) = -1;
model.rxns{end+1} = 'EX_acr-GSH[e]';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Exchange of Acrolein-GSH';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 acr-GSH[e] <=> ';

% % % transport

% cpa[e] --> cpa[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cpa[e]'),end) = -1;
model.S(strcmp(model.mets,'cpa[c]'),end) = 1;
model.rxns{end+1} = 'cpa_transport_e_c';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Cyclophosphamide from extracellular to cytosol';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cpa[e] --> 1 cpa[c]';

% cpa[c] --> cpa[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cpa[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'cpa[e]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = 'cpa_transport_c_e';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1823) | x(3174)';
model.grRules{end+1} = '(5243) or (9429)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1823) = 1;
model.rxnGeneMat(end,3174) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Cyclophosphamide from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cpa[c] + 1 atp[c] + 1 h2o[c] --> 1 cpa[e] + 1 adp[c] + 1 pi[c]';

% dce-cpa[c] <=> dce-cpa[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'dce-cpa[e]'),end) = 1;
model.S(strcmp(model.mets,'dce-cpa[c]'),end) = -1;
model.rxns{end+1} = 'dce-cpa_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Dechloroethylcyclophosphamide from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 dce-cpa[c] <=> 1 dce-cpa[e]';

% caa[c] <=> caa[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'caa[e]'),end) = 1;
model.S(strcmp(model.mets,'caa[c]'),end) = -1;
model.rxns{end+1} = 'caa_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Chloroacetaldehyde from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 caa[c] <=> 1 caa[e]';

% 4k-cpa[c] <=> 4k-cpa[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'4k-cpa[e]'),end) = 1;
model.S(strcmp(model.mets,'4k-cpa[c]'),end) = -1;
model.rxns{end+1} = '4k-cpa_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of 4-Ketocyclophosphamide from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 4k-cpa[c] <=> 1 4k-cpa[e]';

% alcph[c] <=> alcph[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'alcph[e]'),end) = 1;
model.S(strcmp(model.mets,'alcph[c]'),end) = -1;
model.rxns{end+1} = 'alcph_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Alcophosphamide from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 alcph[c] <=> 1 alcph[e]';

% cph[c] <=> cph[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cph[e]'),end) = 1;
model.S(strcmp(model.mets,'cph[c]'),end) = -1;
model.rxns{end+1} = 'cph_transport_c_e';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Carboxyphosphamide from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cph[c] <=> 1 cph[e]';

% phmust-GSH[c] --> phmust-GSH[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'phmust-GSH[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'phmust-GSH[e]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = 'phmust-GSH_transport_c_e';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(331)';
model.grRules{end+1} = '(1244)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,331) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Phosphoramide mustard-GSH from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 phmust-GSH[c] + 1 atp[c] + 1 h2o[c] --> 1 phmust-GSH[e] + 1 adp[c] + 1 pi[c]';

% acr-GSH[c] --> acr-GSH[e]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'acr-GSH[c]'),end) = -1;
model.S(strcmp(model.mets,'atp[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'acr-GSH[e]'),end) = 1;
model.S(strcmp(model.mets,'adp[c]'),end) = 1;
model.S(strcmp(model.mets,'pi[c]'),end) = 1;
model.rxns{end+1} = 'acr-GSH_transport_c_e';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(331)';
model.grRules{end+1} = '(1244)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,331) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Transport of Acrolein-GSH from cytosol to extracellular';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = 'X.X.X.X';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 acr-GSH[c] + 1 atp[c] + 1 h2o[c] --> 1 acr-GSH[e] + 1 adp[c] + 1 pi[c]';

% % % cpa mechanism

% cpa[c] --> 4oh-cpa[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cpa[c]'),end) = -1;
model.S(strcmp(model.mets,'o2[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
model.S(strcmp(model.mets,'h[c]'),end) = -1;
model.S(strcmp(model.mets,'4oh-cpa[c]'),end) = 1;
model.S(strcmp(model.mets,'h2o[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
model.rxns{end+1} = 'cpa_4oh-cpa';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(501) | x(504) | x(502) | x(514)';
model.grRules{end+1} = '(1555) or (1559) or (1557) or (1576)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,501) = 1;
model.rxnGeneMat(end,504) = 1;
model.rxnGeneMat(end,502) = 1;
model.rxnGeneMat(end,514) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Cyclophosphamide to 4-hydroxycyclophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.14.14.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cpa[c] + 1 o2[c] + 1 nadph[c] + 1 h[c] --> 1 4oh-cpa[c] + 1 h2o[c] + 1 nadp[c]';

% 4oh-cpa[e] <=> aldph[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'4oh-cpa[c]'),end) = -1;
model.S(strcmp(model.mets,'aldph[c]'),end) = 1;
model.rxns{end+1} = '4oh-cpa_aldph';
model.lb(end+1) = min(model.lb);
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = '4-hydroxycyclophosphamide to aldophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 4oh-cpa[e] <=> 1 aldph[c]';

% aldph[c] --> phmust[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'aldph[c]'),end) = -1;
model.S(strcmp(model.mets,'phmust[c]'),end) = 1;
model.S(strcmp(model.mets,'acr[c]'),end) = 1;
model.rxns{end+1} = 'aldph_phmust';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} = '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Aldophosphamide to phosphoramide mustard';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 aldph[c] --> 1 phmust[c] + 1 acr[c]';

% % % cpa metabolism

% cpa[c] --> dce-cpa[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'cpa[c]'),end) = -1;
model.S(strcmp(model.mets,'o2[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
model.S(strcmp(model.mets,'h[c]'),end) = -1;
model.S(strcmp(model.mets,'dce-cpa[c]'),end) = 1;
model.S(strcmp(model.mets,'caa[c]'),end) = 1;
model.S(strcmp(model.mets,'h2o[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
model.rxns{end+1} = 'cpa_dce-cpa';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(514) | x(515)';
model.grRules{end+1} = '(1576) or (1577)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,514) = 1;
model.rxnGeneMat(end,515) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Cyclophosphamide to dechloroethylcyclophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.14.14.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 cpa[c] + 1 o2[c] + 1 nadph[c] + 1 h[c] --> 1 dce-cpa[c] + 1 caa[c] + 1 h2o[c] + 1 nadp[c]';

% 4oh-cpa[c] --> 4k-cpa[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'4oh-cpa[c]'),end) = -1;
model.S(strcmp(model.mets,'nad[c]'),end) = -1;
model.S(strcmp(model.mets,'4k-cpa[c]'),end) = 1;
model.S(strcmp(model.mets,'nadh[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = '4oh-cpa_4k-cpa';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(330) | x(339) | x(344) | x(351) | x(355) | x(363) | x(370)';
model.grRules{end+1} = '(124) or (125) or (126) or (127) or (128) or (130) or (131)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,330) = 1;
model.rxnGeneMat(end,339) = 1;
model.rxnGeneMat(end,344) = 1;
model.rxnGeneMat(end,351) = 1;
model.rxnGeneMat(end,355) = 1;
model.rxnGeneMat(end,363) = 1;
model.rxnGeneMat(end,370) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = '4-Hydroxycyclophosphamide to 4-ketocyclophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.1.1.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 4oh-cpa[c] + 1 nad[c] --> 1 4k-cpa[c] + 1 nadh[c] + 1 h[c]';

% aldph[c] --> alcph[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'aldph[c]'),end) = -1;
model.S(strcmp(model.mets,'nadph[c]'),end) = -1;
model.S(strcmp(model.mets,'h[c]'),end) = -1;
model.S(strcmp(model.mets,'alcph[c]'),end) = 1;
model.S(strcmp(model.mets,'nadp[c]'),end) = 1;
model.rxns{end+1} = 'aldph_alcph';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(74) | x(814) | x(2190)';
model.grRules{end+1} = '(10327) or (231) or (57016)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,74) = 1;
model.rxnGeneMat(end,814) = 1;
model.rxnGeneMat(end,2190) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Aldophosphamide to alcophosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.1.1.21';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 aldph[c] + 1 nadph[c] + 1 h[c] --> 1 alcph[c] + 1 nadp[c]';

% aldph[c] --> cph[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'aldph[c]'),end) = -1;
model.S(strcmp(model.mets,'nad[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = -1;
model.S(strcmp(model.mets,'cph[c]'),end) = 1;
model.S(strcmp(model.mets,'nadh[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = 'aldph_cph';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(716) | x(728) | x(2709)';
model.grRules{end+1} = '(216) or (218) or (7915)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,716) = 1;
model.rxnGeneMat(end,728) = 1;
model.rxnGeneMat(end,2709) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Aldophosphamide to carboxyphosphamide';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.2.1.3';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 aldph[c] + 1 nad[c] + 1 h2o[c] --> 1 cph[c] + 1 nadh[c] + 1 h[c]';

% phmust[c] --> phmust-GSH[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'phmust[c]'),end) = -1;
model.S(strcmp(model.mets,'gthrd[c]'),end) = -1;
model.S(strcmp(model.mets,'phmust-GSH[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = 'phmust_phmust-GSH';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1175)';
model.grRules{end+1} = '(2938)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1175) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Phosphoramide mustard to phosphoramide mustard-GSH';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.5.1.18';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 phmust[c] + 1 gthrd[c] --> 1 phmust-GSH + 1 h[c]';

% acr[c] --> acr-GSH[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'acr[c]'),end) = -1;
model.S(strcmp(model.mets,'gthrd[c]'),end) = -1;
model.S(strcmp(model.mets,'acr-GSH[c]'),end) = 1;
model.S(strcmp(model.mets,'h[c]'),end) = 1;
model.rxns{end+1} = 'acr_acr-GSH';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1175)';
model.grRules{end+1} = '(2938)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,1175) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Acrolein to acrolein-GSH';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '2.5.1.18';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'MODULE CYCLOPHOSPHAMIDE';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 acr[c] + 1 gthrd[c] --> 1 acr-GSH + 1 h[c]';

### Redox cont.

Remove wrong genes

In [9]:
% PRDX1 and PRDX2
i = find(strcmp(model.rxns,'GTHP'));
model.rxnGeneMat(i,strcmp(model.genes,'5052')) = 0;
model.rxnGeneMat(i,strcmp(model.genes,'7001')) = 0;
model.rules{i} = 'x(1147) | x(1148) | x(1150)';
model.grRules{i} =  '(2876) or (2877) or (2879)';

% PRDX3
i = find(strcmp(model.rxns,'GTHPm'));
model.rxnGeneMat(i,strcmp(model.genes,'10935')) = 0;
model.rules{i} = 'x(1147) | x(1150)';
model.grRules{i} =  '(2876) or (2879)';

% PRDX6
i = find(strcmp(model.rxns,'HMR_0960'));
model.rxnGeneMat(i,strcmp(model.genes,'9588')) = 0;
model.rules{i} = 'x(954) | x(1147) | x(1148) | x(1149) | x(1150) | x(1151) | x(1152) | x(1619)';
model.grRules{i} =  '(257202) or (2876) or (2877) or (2878) or (2879) or (2880) or (2882) or (493869)';

i = find(strcmp(model.rxns,'HMR_0963'));
model.rxnGeneMat(i,strcmp(model.genes,'9588')) = 0;
model.rules{i} = 'x(954) | x(1147) | x(1148) | x(1149) | x(1150) | x(1151) | x(1152) | x(1619)';
model.grRules{i} =  '(257202) or (2876) or (2877) or (2878) or (2879) or (2880) or (2882) or (493869)';

i = find(strcmp(model.rxns,'HMR_0988'));
model.rxnGeneMat(i,strcmp(model.genes,'9588')) = 0;
model.rules{i} = 'x(954) | x(1147) | x(1148) | x(1149) | x(1150) | x(1151) | x(1152) | x(1619)';
model.grRules{i} =  '(257202) or (2876) or (2877) or (2878) or (2879) or (2880) or (2882) or (493869)';

i = find(strcmp(model.rxns,'HMR_2441'));
model.rxnGeneMat(i,strcmp(model.genes,'9588')) = 0;
model.rules{i} = 'x(954) | x(1147) | x(1148) | x(1149) | x(1150) | x(1151) | x(1152) | x(1619)';
model.grRules{i} =  '(257202) or (2876) or (2877) or (2878) or (2879) or (2880) or (2882) or (493869)';

i = find(strcmp(model.rxns,'HMR_1048'));
model.rxnGeneMat(i,strcmp(model.genes,'9588')) = 0;
model.rules{i} = 'x(954) | x(1147) | x(1148) | x(1149) | x(1150) | x(1151) | x(1152) | x(1619)';
model.grRules{i} =  '(257202) or (2876) or (2877) or (2878) or (2879) or (2880) or (2882) or (493869)';

i = find(strcmp(model.rxns,'HMR_1049'));
model.rxnGeneMat(i,strcmp(model.genes,'9588')) = 0;
model.rules{i} = 'x(954) | x(1147) | x(1148) | x(1149) | x(1150) | x(1151) | x(1152) | x(1619)';
model.grRules{i} =  '(257202) or (2876) or (2877) or (2878) or (2879) or (2880) or (2882) or (493869)';

Metabolites

In [10]:
% Pr-SH[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'Pr-SH[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'X';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Protein, reduced';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% Pr-SSG[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'Pr-SSG[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'X';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Protein, glutathionylated';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% Pr-SS[c]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'Pr-SS[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'X';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Protein, oxidized';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% Pr-SH[m]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'Pr-SH[m]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'X';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Protein, reduced';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% Pr-SSG[m]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'Pr-SSG[c]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'X';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Protein, glutathionylated';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

% Pr-SS[m]
model.S(end+1,:) = zeros(1,length(model.rxns));
model.mets{end+1} = 'Pr-SS[m]';
model.b(end+1) = 0;
model.csense(end+1) = 'E';
model.metCharges(end+1) = 0;
model.metFormulas{end+1} = 'X';
model.metSmiles{end+1} = '';
model.metNames{end+1} = 'Protein, oxidized';
model.metHMDBID{end+1} = '';
model.metInChIString{end+1} = '';
model.metKEGGID{end+1} = '';
model.metPubChemID{end+1} = '';
model.metCHEBIID{end+1} = '';
model.metPdMap{end+1} = '';
model.metReconMap{end+1} = '';

Reactions

In [11]:
% 1 h2o2[c] + 1 trdrd[c] --> 2 h2o[c] + 1 trdox[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'h2o2[c]'),end) = -1;
model.S(strcmp(model.mets,'trdrd[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = 2;
model.S(strcmp(model.mets,'trdox[c]'),end) = 1;
model.rxns{end+1} = 'Prx_c';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1646) | x(2590)';
model.grRules{end+1} =  '(5052) or (7001)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,strcmp(model.genes,'5052')) = 1;
model.rxnGeneMat(end,strcmp(model.genes,'7001')) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Peroxiredoxin, cytoplasmic';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.11.1.15';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 h2o2[c] + 1 trdrd[c] --> 2 h2o[c] + 1 trdox[c]';

% 1 h2o2[m] + 1 trdrd[m] --> 2 h2o[m] + 1 trdox[m]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'h2o2[m]'),end) = -1;
model.S(strcmp(model.mets,'trdrd[m]'),end) = -1;
model.S(strcmp(model.mets,'h2o[m]'),end) = 2;
model.S(strcmp(model.mets,'trdox[m]'),end) = 1;
model.rxns{end+1} = 'Prx_m';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(168)';
model.grRules{end+1} =  '(10935)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,strcmp(model.genes,'10935')) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Peroxiredoxin, mitochondrial';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.11.1.15';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 h2o2[m] + 1 trdrd[m] --> 2 h2o[m] + 1 trdox[m]';

% 1 h2o2[c] + 1 Pr-SH[c] + 1 gthrd[c] --> 2 h2o[c] + 1 Pr-SSG[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'h2o2[c]'),end) = -1;
model.S(strcmp(model.mets,'Pr-SH[c]'),end) = -1;
model.S(strcmp(model.mets,'gthrd[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = 2;
model.S(strcmp(model.mets,'Pr-SSG[c]'),end) = 1;
model.rxns{end+1} = 'Protein_thiol_SSG';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} =  '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Protein thiol oxidation, SSG';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 h2o2[c] + 1 Pr-SH[c] + 1 gthrd[c] --> 2 h2o[c] + 1 Pr-SSG[c]';

% 1 Pr-SSG[c] + 1 gthrd[c] --> 1 Pr-SH[c] + 1 gthox[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'Pr-SSG[c]'),end) = -1;
model.S(strcmp(model.mets,'gthrd[c]'),end) = -1;
model.S(strcmp(model.mets,'Pr-SH[c]'),end) = 1;
model.S(strcmp(model.mets,'gthox[c]'),end) = 1;
model.rxns{end+1} = 'Grx_c';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1096)';
model.grRules{end+1} =  '(2745)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,strcmp(model.genes,'2745')) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Glutaredoxin, mitochondrial';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.20.4.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 Pr-SSG[c] + 1 gthrd[c] --> 1 Pr-SH[c] + 1 gthox[c]';

% 1 h2o2[c] + 1 Pr-SH[c] --> 2 h2o[c] + 1 Pr-SS[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'h2o2[c]'),end) = -1;
model.S(strcmp(model.mets,'Pr-SH[c]'),end) = -1;
model.S(strcmp(model.mets,'h2o[c]'),end) = 2;
model.S(strcmp(model.mets,'Pr-SS[c]'),end) = 1;
model.rxns{end+1} = 'Protein_thiol_SS';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} =  '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Protein thiol oxidation, SS';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 h2o2[c] + 1 Pr-SH[c] --> 2 h2o[c] + 1 Pr-SS[c]';

% 1 Pr-SS[c] + 1 trdrd[c] --> 1 Pr-SH[c] + 1 trdox[c]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'Pr-SS[c]'),end) = -1;
model.S(strcmp(model.mets,'trdrd[c]'),end) = -1;
model.S(strcmp(model.mets,'Pr-SH[c]'),end) = 1;
model.S(strcmp(model.mets,'trdox[c]'),end) = 1;
model.rxns{end+1} = 'Protein_thiol_reduction_c';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} =  '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Protein thiol reduction by thioredoxin';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 Pr-SS[c] + 1 trdrd[c] --> 1 Pr-SH[c] + 1 trdox[c]';

% 1 h2o2[m] + 1 Pr-SH[m] + 1 gthrd[m] --> 2 h2o[m] + 1 Pr-SSG[m]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'h2o2[m]'),end) = -1;
model.S(strcmp(model.mets,'Pr-SH[m]'),end) = -1;
model.S(strcmp(model.mets,'gthrd[m]'),end) = -1;
model.S(strcmp(model.mets,'h2o[m]'),end) = 2;
model.S(strcmp(model.mets,'Pr-SSG[m]'),end) = 1;
model.rxns{end+1} = 'Protein_thiol_SSG_m';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} =  '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Protein thiol oxidation, SSG';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 h2o2[m] + 1 Pr-SH[m] + 1 gthrd[m] --> 2 h2o[m] + 1 Pr-SSG[m]';

% 1 Pr-SSG[m] + 1 gthrd[m] --> 1 Pr-SH[m] + 1 gthox[m]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'Pr-SSG[m]'),end) = -1;
model.S(strcmp(model.mets,'gthrd[m]'),end) = -1;
model.S(strcmp(model.mets,'Pr-SH[m]'),end) = 1;
model.S(strcmp(model.mets,'gthox[m]'),end) = 1;
model.rxns{end+1} = 'Grx_m';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = 'x(1670)';
model.grRules{end+1} =  '(51022)';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnGeneMat(end,strcmp(model.genes,'51022')) = 1;
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Glutaredoxin, mitochondrial';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '1.20.4.1';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 Pr-SSG[m] + 1 gthrd[m] --> 1 Pr-SH[m] + 1 gthox[m]';

% 1 h2o2[m] + 1 Pr-SH[m] --> 2 h2o[m] + 1 Pr-SS[m]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'h2o2[m]'),end) = -1;
model.S(strcmp(model.mets,'Pr-SH[m]'),end) = -1;
model.S(strcmp(model.mets,'h2o[m]'),end) = 2;
model.S(strcmp(model.mets,'Pr-SS[m]'),end) = 1;
model.rxns{end+1} = 'Protein_thiol_SS_m';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} =  '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Protein thiol oxidation, SS';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 h2o2[m] + 1 Pr-SH[m] --> 2 h2o[m] + 1 Pr-SS[m]';

% 1 Pr-SS[m] + 1 trdrd[m] --> 1 Pr-SH[m] + 1 trdox[m]
model.S(:,end+1) = zeros(length(model.mets),1);
model.S(strcmp(model.mets,'Pr-SS[m]'),end) = -1;
model.S(strcmp(model.mets,'trdrd[m]'),end) = -1;
model.S(strcmp(model.mets,'Pr-SH[m]'),end) = 1;
model.S(strcmp(model.mets,'trdox[m]'),end) = 1;
model.rxns{end+1} = 'Protein_thiol_reduction_m';
model.lb(end+1) = 0;
model.ub(end+1) = max(model.ub);
model.c(end+1) = 0;
model.rules{end+1} = '';
model.grRules{end+1} =  '';
model.rxnGeneMat(end+1,:) = zeros(1,length(model.genes));
model.rxnConfidenceScores(end+1) = 0;
model.rxnNames{end+1} = 'Protein thiol reduction by thioredoxin, mitochondrial';
model.rxnNotes{end+1} = '';
model.rxnECNumbers{end+1} = '';
model.rxnReferences{end+1} = '';
model.rxnKEGGID{end+1} = '';
model.subSystems{end+1} = 'ROS detoxification';
model.rxnCOG{end+1} = '';
model.rxnKeggOrthology{end+1} = '';
model.rxnReconMap{end+1} = '';
model.rxnFormulas{end+1} = '1 Pr-SS[m] + 1 trdrd[m] --> 1 Pr-SH[m] + 1 trdox[m]';

### Catalase reactions involving NADPH

In [9]:
% fix CATr genes and EC number
model.rules{strcmp(model.rxns,'CATr')} = model.rules{strcmp(model.rxns,'r0010')};
model.grRules{strcmp(model.rxns,'CATr')} = model.grRules{strcmp(model.rxns,'r0010')};
model.rxnGeneMat(strcmp(model.rxns,'CATr'),:) = model.rxnGeneMat(strcmp(model.rxns,'r0010'),:);
model.rxnECNumbers{strcmp(model.rxns,'CATr')} = model.rxnECNumbers{strcmp(model.rxns,'r0010')};

In [22]:
reaction_names = {'r0010','CATm','CATp','CATr'};
compartment_names = {'[c]','[m]','[x]','[r]'};
for i = 1:length(reaction_names)
    model.S(:,end+1) = model.S(:,strcmp(model.rxns,reaction_names{i}));
    model.S(strcmp(model.mets,strcat('nadph',compartment_names{i})),end) = -0.1282;
    model.S(strcmp(model.mets,strcat('nadp',compartment_names{i})),end) = 0.1282;
    model.S(strcmp(model.mets,strcat('h',compartment_names{i})),end) = 0.1282;
    model.rxns{end+1} = strcat(reaction_names{i},'_nadph');
    model.lb(end+1) = model.lb(strcmp(model.rxns,reaction_names{i}));
    model.ub(end+1) = model.ub(strcmp(model.rxns,reaction_names{i}));
    model.c(end+1) = model.c(strcmp(model.rxns,reaction_names{i}));
    model.rules{end+1} = model.rules{strcmp(model.rxns,reaction_names{i})};
    model.grRules{end+1} =  model.grRules{strcmp(model.rxns,reaction_names{i})};
    model.rxnGeneMat(end+1,:) = model.rxnGeneMat(strcmp(model.rxns,reaction_names{i}),:);
    model.rxnConfidenceScores(end+1) = model.rxnConfidenceScores(strcmp(model.rxns,reaction_names{i}));
    model.rxnNames{end+1} = model.rxnNames{strcmp(model.rxns,reaction_names{i})};
    model.rxnNotes{end+1} = model.rxnNotes{strcmp(model.rxns,reaction_names{i})};
    model.rxnECNumbers{end+1} = model.rxnECNumbers{strcmp(model.rxns,reaction_names{i})};
    model.rxnReferences{end+1} = model.rxnReferences{strcmp(model.rxns,reaction_names{i})};
    model.rxnKEGGID{end+1} = model.rxnKEGGID{strcmp(model.rxns,reaction_names{i})};
    model.subSystems{end+1} = model.subSystems{strcmp(model.rxns,reaction_names{i})};
    model.rxnCOG{end+1} = model.rxnCOG{strcmp(model.rxns,reaction_names{i})};
    model.rxnKeggOrthology{end+1} = model.rxnKeggOrthology{strcmp(model.rxns,reaction_names{i})};
    model.rxnReconMap{end+1} = model.rxnReconMap{strcmp(model.rxns,reaction_names{i})};
    model.rxnFormulas{end+1} = strcat(model.rxnFormulas{strcmp(model.rxns,reaction_names{i})}(1:9),' + 0.1282 nadph',compartment_names{i},model.rxnFormulas{strcmp(model.rxns,reaction_names{i})}(10:end),' + 0.1282 nadp',compartment_names{i},' + 0.1282 h',compartment_names{i});
end

### Save model file 

In [64]:
#load('recon3d_qflux.mat','model');

In [74]:
save('recon3d_qflux.mat','model');

### Create information files

Create reaction information file

In [75]:
f = fopen('reactions.tsv','w');
fprintf(f,'RXNID\tNAME\tFORMULA\tSUBSYSTEM\tGR RULE\tEC NUMBER\tKEGG\tNOTES\tREFERENCES\n');
for i = 1:length(model.rxns)
    fprintf(f,'%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n',model.rxns{i},model.rxnNames{i},model.rxnFormulas{i},model.subSystems{i},model.grRules{i},model.rxnECNumbers{i},model.rxnKEGGID{i},model.rxnNotes{i},model.rxnReferences{i});
end
fclose(f);

Create metabolite information file

In [76]:
f = fopen('metabolites.tsv','w');
fprintf(f,'METID\tNAME\tFORMULA\tCHARGE\tCHEBI\tKEGG\tPUBCHEM\tHMDB\tINCHI STRING\tSMILE\n');
for i = 1:length(model.mets)
    fprintf(f,'%s\t%s\t%s\t%d\t%s\t%s\t%s\t%s\t%s\t%s\n',model.mets{i},model.metNames{i},model.metFormulas{i},model.metCharges(i),model.metCHEBIID{i},model.metKEGGID{i},model.metPubChemID{i},model.metHMDBID{i},model.metInChIString{i},model.metSmiles{i});
end
fclose(f);

Create gene information file

In [77]:
f = fopen('genes.tsv','w');
fprintf(f,'GENEID\tSYMBOL\n');
for i = 1:length(model.genes)
    fprintf(f,'%s\t%s\n',model.genes{i},model.geneSymbols{i});
end
fclose(f);